### Section 1 — Chargement & normalisation (L1 + L3) + contrôles de cohérence

In [ ]:
# -*- coding: utf-8 -*-
"""
Section 1 — Chargement & normalisation (L1 + L3) + contrôles de cohérence

Objectifs (Section 1)
1) Lire L1 (2 onglets) + L3
2) Normaliser les champs (IDs, niveaux 1..5, heures totales)
3) Contrôles qualité: valeurs manquantes, distributions, cohérences simples
4) Produire des artefacts "clean" (CSV) pour les pages suivantes
"""

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import re
import numpy as np
import pandas as pd

# ============================================================
# 0) CONFIG — adapte si besoin
# ============================================================
L1_PATH = "./data/2025-08/L1.20250818-DataMathsElysa.xlsx"
print(os.getcwd())
# L1_PATH = "../../" + L1_PATH
L3_PATH = "./data/2025-08/L3.features_by_student.byPretest.csv"

OUT_DIR = "./out/202602/out_pomdp"
os.makedirs(OUT_DIR, exist_ok=True)

def out_path(fname: str) -> str:
    return os.path.join(OUT_DIR, f"L0-{fname}")

# Hypothèses  (données fournies par toi)
HOURS_PER_ACTIVITY = 2.0   # 1 activité ≈ 2h (vérité absolue dans l'analyse)
ACTIVITIES_PER_DAY = 4     # 4 activités / jour (positions 1..4)
HOURS_PER_DAY = HOURS_PER_ACTIVITY * ACTIVITIES_PER_DAY  # 8h / jour

assert os.path.exists(L1_PATH), f"Introuvable: {L1_PATH}"
assert os.path.exists(L3_PATH), f"Introuvable: {L3_PATH}"

xls = pd.ExcelFile(L1_PATH)
print("L1 — Onglets détectés:", xls.sheet_names)

# Lire les deux onglets (on n'impose pas le nom exact, on mappe par heuristique)
df_sheets = {name: pd.read_excel(L1_PATH, sheet_name=name) for name in xls.sheet_names}

# Heuristique: onglet "Résultatsélèves" = contient Pretest/Final + IDélèves
# onglet "Activités" = contient Semaine/Jour + colonnes activités 0..4
def _score_results_sheet(d: pd.DataFrame) -> int:
    cols = {c.strip(): c for c in d.columns}
    score = 0
    for k in ["IDélèves","Pretest","Final","Age","Genre","Zone"]:
        if k in cols: score += 2
    if "Nombre d'heures de remédiation fait au total" in cols: score += 2
    return score

def _score_activities_sheet(d: pd.DataFrame) -> int:
    cols = [str(c) for c in d.columns]
    score = 0
    if any(c.lower().strip() == "semaine" for c in cols): score += 2
    if any(c.lower().strip() == "jour" for c in cols): score += 2
    # beaucoup de colonnes après les 2 premières
    if len(cols) > 20: score += 2
    return score

sheet_scores = []
for name, d in df_sheets.items():
    sheet_scores.append((name, _score_results_sheet(d), _score_activities_sheet(d)))

# Choix
results_sheet = sorted(sheet_scores, key=lambda t: t[1], reverse=True)[0][0]
activities_sheet = sorted(sheet_scores, key=lambda t: t[2], reverse=True)[0][0]

df_results = df_sheets[results_sheet].copy()
df_acts = df_sheets[activities_sheet].copy()
df_l3 = pd.read_csv(L3_PATH)

print("\n L1 (résultats) choisi:", results_sheet, "| shape:", df_results.shape)
print(" L1 (activités) choisi:", activities_sheet, "| shape:", df_acts.shape)
print(" L3 chargé:", "| shape:", df_l3.shape)

# ============================================================
# 2) NORMALISATION DES COLONNES (L1 Résultats)
# ============================================================
def _norm_colname(s: str) -> str:
    s = str(s).strip()
    s = s.replace("\n", " ").replace("\r", " ")
    s = re.sub(r"\s+", " ", s)
    return s

df_results.columns = [_norm_colname(c) for c in df_results.columns]
df_acts.columns = [_norm_colname(c) for c in df_acts.columns]
df_l3.columns = [_norm_colname(c) for c in df_l3.columns]

# Renommage standard
rename_results = {
    "IDélèves": "StudentID",
    "Nombre d'heures de remédiation fait au total": "HoursTotal_L1",
    "Pretest": "Pretest",
    "Final": "Final",
    "Age": "Age",
    "Classe": "Classe",
    "Genre": "Genre",
    "Zone": "Zone",
}
for k, v in rename_results.items():
    if k in df_results.columns:
        df_results.rename(columns={k: v}, inplace=True)

# Coercitions types
if "StudentID" in df_results.columns:
    df_results["StudentID"] = df_results["StudentID"].astype(str).str.strip()

for c in ["Age", "Pretest", "Final", "HoursTotal_L1"]:
    if c in df_results.columns:
        df_results[c] = pd.to_numeric(df_results[c], errors="coerce")

# bornage niveaux 1..5
def cap_level(x):
    try:
        v = int(x)
    except Exception:
        return np.nan
    return min(5, max(1, v))

if "Pretest" in df_results.columns:
    df_results["Pretest_i"] = df_results["Pretest"].apply(cap_level)
if "Final" in df_results.columns:
    df_results["Final_i"] = df_results["Final"].apply(cap_level)

# ============================================================
# 3) NORMALISATION (L1 Activités)
# ============================================================
# Attendu: colonnes "Semaine", "Jour", puis activités groupées par préfixes (Deb, Unchiffe, Deuxchiffres, TroisA, TroisB)
# Standardiser "Semaine" et "Jour"
col_week = None
col_day = None
for c in df_acts.columns:
    cl = c.lower().strip()
    if cl == "semaine": col_week = c
    if cl == "jour": col_day = c

if col_week is None or col_day is None:
    raise ValueError("Onglet Activités: colonnes 'Semaine' et/ou 'Jour' introuvables après normalisation.")

df_acts.rename(columns={col_week: "Week", col_day: "Day"}, inplace=True)
df_acts["Week"] = pd.to_numeric(df_acts["Week"], errors="coerce")
df_acts["Day"] = pd.to_numeric(df_acts["Day"], errors="coerce")

# Colonnes d'activités = tout sauf Week/Day
activity_cols = [c for c in df_acts.columns if c not in ["Week","Day"]]

# Forcer numériques (0..4)
for c in activity_cols:
    df_acts[c] = pd.to_numeric(df_acts[c], errors="coerce")

# Détecter les préfixes bloc (avant le premier espace)
def block_prefix(col: str) -> str:
    s = str(col).strip()
    # ex: "Deb Lecture de la Table d’addition"
    return s.split(" ", 1)[0] if " " in s else s

df_blocks = pd.Series([block_prefix(c) for c in activity_cols], name="Block")
block_counts = df_blocks.value_counts().to_dict()

print("\nBlocs détectés (préfixes) dans L1/Activités:")
for k, v in sorted(block_counts.items(), key=lambda kv: (-kv[1], kv[0])):
    print(f" - {k}: {v} colonnes")

# Marquer bloc pour chaque colonne activité
col_to_block = {c: block_prefix(c) for c in activity_cols}

# ============================================================
# 4) CONTROLES METIER (L1 Activités)
# ============================================================
# 4.1 Vérifier que chaque jour contient exactement 4 activités exécutées (valeurs 1..4)
def count_executed_per_day(row: pd.Series) -> int:
    vals = row[activity_cols].values
    return int(np.sum(np.isin(vals, [1,2,3,4])))

exec_counts = df_acts.apply(count_executed_per_day, axis=1)
print("\nContrôle: nb d'activités exécutées par jour (attendu = 4):")
print(exec_counts.describe())

bad_days = df_acts.loc[exec_counts != 4, ["Week","Day"]].copy()
bad_days["ExecutedCount"] = exec_counts[exec_counts != 4].values
print(f"Jours anormaux (ExecutedCount != 4): {len(bad_days)}")
if len(bad_days) > 0:
    print(bad_days.head(20).to_string(index=False))

# 4.2 Vérifier présence des positions 1..4 dans chaque ligne (si 4 activités, on veut idéalement 1,2,3,4 chacune une fois)
def missing_positions(row: pd.Series):
    vals = row[activity_cols].values
    pos = set([int(v) for v in vals if v in [1,2,3,4]])
    missing = [p for p in [1,2,3,4] if p not in pos]
    return missing

miss_pos = df_acts.apply(missing_positions, axis=1)
n_miss = miss_pos.apply(len)
print("\nContrôle: positions manquantes dans la journée (attendu = 0):")
print(n_miss.value_counts().sort_index().to_string())

# ============================================================
# 5) CONTROLES (L1 Résultats) + (L3)
# ============================================================
print("\nL1 Résultats — aperçu colonnes:", df_results.columns.tolist())

# Checks simples niveaux
if "Pretest_i" in df_results.columns:
    print("\nDistribution Pretest_i (L1):")
    print(df_results["Pretest_i"].value_counts(dropna=False).sort_index().to_string())

if "Final_i" in df_results.columns:
    print("\nDistribution Final_i (L1):")
    print(df_results["Final_i"].value_counts(dropna=False).sort_index().to_string())

if "HoursTotal_L1" in df_results.columns:
    print("\nHoursTotal_L1 — stats:")
    print(df_results["HoursTotal_L1"].describe().to_string())

# L3 checks
print("\nL3 — colonnes importantes présentes ?")
for c in ["HoursTotal","Pretest","Final","Delta","Mastery_ge4","Age","Genre","Zone","LevelTag"]:
    print(f" - {c}: {'OK' if c in df_l3.columns else 'ABSENT'}")

if "HoursTotal" in df_l3.columns:
    df_l3["HoursTotal"] = pd.to_numeric(df_l3["HoursTotal"], errors="coerce")
    print("\nHoursTotal (L3) — stats:")
    print(df_l3["HoursTotal"].describe().to_string())

# ============================================================
# 6) COHERENCE L1 vs L3 : heures totales
# ============================================================
# Si L3 a StudentID => merge direct. Sinon: on garde juste des comparaisons de distributions.
has_id_l3 = any(c.lower() in ["studentid","idélèves","ideleves","id_eleve","id"] for c in df_l3.columns)

if has_id_l3:
    # repérer la colonne id la plus plausible
    cand = None
    for c in df_l3.columns:
        if c.lower() in ["studentid","idélèves","ideleves","id_eleve","id"]:
            cand = c
            break
    if cand and "StudentID" in df_results.columns:
        df_l3["_StudentID"] = df_l3[cand].astype(str).str.strip()
        df_merge = df_results.merge(df_l3, left_on="StudentID", right_on="_StudentID", how="inner", suffixes=("_L1","_L3"))
        print("\nMerge L1-L3 via StudentID:", df_merge.shape)

        if "HoursTotal_L1" in df_merge.columns and "HoursTotal" in df_merge.columns:
            diff = (df_merge["HoursTotal"] - df_merge["HoursTotal_L1"])
            print("\nCohérence HoursTotal: (L3 - L1) stats")
            print(diff.describe().to_string())
else:
    print("\nInfo: L3 ne contient pas d'ID explicite (StudentID). On ne merge pas L1 et L3 à cette étape.")

df_results.to_csv(out_path("L1_results_clean.csv"), index=False, encoding="utf-8-sig")
df_acts.to_csv(out_path("L1_activities_clean.csv"), index=False, encoding="utf-8-sig")

df_colblock = pd.DataFrame({
    "ActivityColumn": activity_cols,
    "Block": [col_to_block[c] for c in activity_cols]
})
df_colblock.to_csv(out_path("L1_activity_column_blocks.csv"), index=False, encoding="utf-8-sig")

summary = {
    "L1_path": L1_PATH,
    "L3_path": L3_PATH,
    "L1_results_sheet": results_sheet,
    "L1_activities_sheet": activities_sheet,
    "L1_results_shape": df_results.shape,
    "L1_activities_shape": df_acts.shape,
    "L3_shape": df_l3.shape,
    "HOURS_PER_ACTIVITY": HOURS_PER_ACTIVITY,
    "ACTIVITIES_PER_DAY": ACTIVITIES_PER_DAY,
    "HOURS_PER_DAY": HOURS_PER_DAY,
    "blocks_detected": block_counts,
    "n_bad_days_executedcount_ne_4": int((exec_counts != 4).sum()),
    "n_days_missing_positions": int((n_miss > 0).sum()),
    "outputs": {
        "L1_results_clean": out_path("L1_results_clean.csv"),
        "L1_activities_clean": out_path("L1_activities_clean.csv"),
        "L1_activity_column_blocks": out_path("L1_activity_column_blocks.csv"),
    }
}
with open(out_path("SECTION1_summary.json"), "w", encoding="utf-8") as f:
    import json
    json.dump(summary, f, ensure_ascii=False, indent=2)

print("\n Section 1 terminée.")
print("Fichiers :")
print(" -", out_path("L1_results_clean.csv"))
print(" -", out_path("L1_activities_clean.csv"))
print(" -", out_path("L1_activity_column_blocks.csv"))
print(" -", out_path("SECTION1_summary.json"))

C:\Data-Mahefa\thesis\PhD\Articles-Mahefa\articles-elysa
L1 — Onglets détectés: ['Résultatsélèves', 'Activités']

✅ L1 (résultats) choisi: Résultatsélèves | shape: (813, 8)
✅ L1 (activités) choisi: Activités | shape: (10, 54)
✅ L3 chargé: | shape: (813, 265)

Blocs détectés (préfixes) dans L1/Activités:
 - Deuxchiffres: 13 colonnes
 - TroisA: 11 colonnes
 - Deb: 10 colonnes
 - Unchiffe: 10 colonnes
 - TroisB: 8 colonnes

Contrôle: nb d'activités exécutées par jour (attendu = 4):
count    10.000000
mean     20.200000
std       0.632456
min      20.000000
25%      20.000000
50%      20.000000
75%      20.000000
max      22.000000
dtype: float64
Jours anormaux (ExecutedCount != 4): 10
 Week  Day  ExecutedCount
    1    1             20
    1    2             20
    1    3             20
    1    4             20
    1    5             20
    2    6             20
    2    7             20
    2    8             22
    2    9             20
    2   10             20

Contrôle: positions ma

### Section 1 — Reconstruction des séquences journalières depuis L1 (onglet Activités)

In [ ]:
# -*- coding: utf-8 -*-
"""
Section 1 — Reconstruction des séquences journalières depuis L1 (onglet Activités)
+ regroupement par “blocs/niveaux” (préfixes de colonnes)
+ extraction d’une séquence ordonnée (pos1..pos4) par jour et par bloc
+ production d’artefacts réutilisables pour l’apprentissage (section suivantes)

Objectif
- Dans L1/Activités : chaque ligne = un jour (Week, Day)
- Les colonnes “activités” contiennent des valeurs 0..4 :
  0 = non exécutée ce jour
  1..4 = position d’exécution (4 activités exécutées/jour)
- Les activités sont regroupées par “bloc” via le préfixe (ex: Deb, Unchiffre, Deuxchiffres, TroisA, TroisB, …)
- On reconstruit pour chaque (Week,Day,Bloc) la séquence journalière :
    pos1 = activité dont la valeur=1
    pos2 = activité dont la valeur=2
    pos3 = activité dont la valeur=3
    pos4 = activité dont la valeur=4
"""

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import re
import json
import numpy as np
import pandas as pd

# ============================================================
# 0) CONFIG — reprend la sortie de la Section 1
# ============================================================
OUT_DIR = "./out/202602/out_pomdp"
os.makedirs(OUT_DIR, exist_ok=True)

def out_path(fname: str) -> str:
    return os.path.join(OUT_DIR, f"L0-{fname}")

L1_ACTS_CLEAN = out_path("L1_activities_clean.csv")
L1_COLBLOCK   = out_path("L1_activity_column_blocks.csv")

assert os.path.exists(L1_ACTS_CLEAN), "Exécute d’abord la Section 1 (L1_activities_clean.csv introuvable)."
assert os.path.exists(L1_COLBLOCK),   "Exécute d’abord la Section 1 (L1_activity_column_blocks.csv introuvable)."

df_acts = pd.read_csv(L1_ACTS_CLEAN)
df_colblock = pd.read_csv(L1_COLBLOCK)

# Attendu: colonnes Week/Day
assert "Week" in df_acts.columns and "Day" in df_acts.columns, "Week/Day introuvables dans L1_activities_clean.csv"

# ============================================================
# 1) IDENTIFIER BLOCS + ACTIVITÉS
# ============================================================
activity_cols = df_colblock["ActivityColumn"].astype(str).tolist()
col_to_block = dict(zip(df_colblock["ActivityColumn"].astype(str), df_colblock["Block"].astype(str)))

blocks = sorted(df_colblock["Block"].unique().tolist())
print("Blocs détectés:", blocks)

# Activités par bloc
block_to_cols = {b: [c for c in activity_cols if col_to_block.get(c) == b] for b in blocks}

# Fonction: extraire “nom d’activité” (sans préfixe bloc)
def activity_name(col: str) -> str:
    s = str(col).strip()
    # retire le préfixe "Bloc "
    # ex: "Deb Lecture du Tableau de nombres" -> "Lecture du Tableau de nombres"
    parts = s.split(" ", 1)
    return parts[1].strip() if len(parts) > 1 else s

block_to_activities = {b: [activity_name(c) for c in block_to_cols[b]] for b in blocks}

# ============================================================
# 2) RECONSTRUIRE SEQUENCE (pos1..pos4) POUR UN JOUR ET UN BLOC
# ============================================================
def extract_day_sequence(row: pd.Series, cols_for_block: list):
    """
    Retourne:
      - seq = {pos1:act, pos2:act, pos3:act, pos4:act}
      - quality flags (missing positions, duplicate positions, etc.)
    """
    # valeurs dans les colonnes du bloc
    vals = row[cols_for_block].values
    # map position -> indices d’activités (peut être 0, 1 ou plusieurs si data anormale)
    pos_to_idx = {p: np.where(vals == p)[0].tolist() for p in [1,2,3,4]}

    seq = {}
    issues = []
    for p in [1,2,3,4]:
        idxs = pos_to_idx[p]
        if len(idxs) == 0:
            seq[f"pos{p}"] = None
            issues.append(f"missing_pos{p}")
        elif len(idxs) > 1:
            # anomalie: plusieurs activités marquées même position
            seq[f"pos{p}"] = activity_name(cols_for_block[idxs[0]])
            issues.append(f"multi_pos{p}")
        else:
            seq[f"pos{p}"] = activity_name(cols_for_block[idxs[0]])

    # contrôle: nb activités exécutées (= count of vals in {1,2,3,4})
    executed = int(np.sum(np.isin(vals, [1,2,3,4])))
    if executed != 4:
        issues.append(f"executed_count={executed}")

    return seq, issues

# ============================================================
# 3) CONSTRUIRE TABLES LONGUES (par jour, par bloc)
# ============================================================
rows_long = []
rows_list = []
rows_vector = []

for _, r in df_acts.iterrows():
    wk = int(r["Week"]) if pd.notna(r["Week"]) else None
    dy = int(r["Day"]) if pd.notna(r["Day"]) else None

    for b in blocks:
        cols_b = block_to_cols[b]
        if not cols_b:
            continue

        seq, issues = extract_day_sequence(r, cols_b)

        # représentation liste ordonnée [pos1,pos2,pos3,pos4]
        seq_list = [seq["pos1"], seq["pos2"], seq["pos3"], seq["pos4"]]

        # représentation “vecteur 0..4” sur toutes les activités du bloc
        # (même logique que ton format Day i: [0,4,0,2,...])
        # chaque entrée = 0 si non exécutée, sinon position 1..4
        v = []
        for c in cols_b:
            x = r[c]
            if pd.isna(x):
                v.append(0)
            else:
                xi = int(x)
                v.append(xi if xi in [1,2,3,4] else 0)

        rows_long.append({
            "Week": wk, "Day": dy, "Block": b,
            "pos1": seq["pos1"], "pos2": seq["pos2"], "pos3": seq["pos3"], "pos4": seq["pos4"],
            "Issues": "|".join(issues) if issues else ""
        })

        rows_list.append({
            "Week": wk, "Day": dy, "Block": b,
            "Sequence": json.dumps(seq_list, ensure_ascii=False),
            "Issues": "|".join(issues) if issues else ""
        })

        # vector row (wide)
        row_vec = {"Week": wk, "Day": dy, "Block": b, "Issues": "|".join(issues) if issues else ""}
        # noms de colonnes lisibles
        for i, c in enumerate(cols_b, start=1):
            row_vec[f"act_{i}:{activity_name(c)}"] = v[i-1]
        rows_vector.append(row_vec)

df_long = pd.DataFrame(rows_long)
df_list = pd.DataFrame(rows_list)
df_vec  = pd.DataFrame(rows_vector)

# ============================================================
# 4) STATS QUALITE PAR BLOC
# ============================================================
def issues_stats(df_long_block: pd.DataFrame):
    issues = df_long_block["Issues"].fillna("").astype(str)
    total = len(df_long_block)
    n_ok = int((issues == "").sum())
    n_bad = total - n_ok

    # compter chaque type d’issue
    counter = {}
    for s in issues:
        if not s:
            continue
        parts = s.split("|")
        for p in parts:
            counter[p] = counter.get(p, 0) + 1

    return {"total_days": total, "ok": n_ok, "bad": n_bad, "issue_counts": counter}

block_quality = {b: issues_stats(df_long[df_long["Block"] == b]) for b in blocks}

print("\nQualité par bloc (résumé):")
for b in blocks:
    q = block_quality[b]
    print(f" - {b}: total={q['total_days']} ok={q['ok']} bad={q['bad']}")

df_long.to_csv(out_path("SECTION1_L1_sequences_by_block_long.csv"), index=False, encoding="utf-8-sig")
df_list.to_csv(out_path("SECTION2_L1_sequences_by_block_list.csv"), index=False, encoding="utf-8-sig")
df_vec.to_csv(out_path("SECTION2_L1_sequences_by_block_vector.csv"), index=False, encoding="utf-8-sig")

with open(out_path("SECTION2_block_to_activities.json"), "w", encoding="utf-8") as f:
    json.dump(block_to_activities, f, ensure_ascii=False, indent=2)

with open(out_path("SECTION2_block_quality.json"), "w", encoding="utf-8") as f:
    json.dump(block_quality, f, ensure_ascii=False, indent=2)

print("\n Section 1 terminée.")
print("Fichiers :")
print(" -", out_path("SECTION2_L1_sequences_by_block_long.csv"))
print(" -", out_path("SECTION2_L1_sequences_by_block_list.csv"))
print(" -", out_path("SECTION2_L1_sequences_by_block_vector.csv"))
print(" -", out_path("SECTION2_block_to_activities.json"))
print(" -", out_path("SECTION2_block_quality.json"))

# ============================================================
# 6) Afficher 3 jours pour chaque bloc
# ============================================================
print("\nAperçu  (3 premiers jours par bloc) :")
for b in blocks:
    sub = df_long[df_long["Block"] == b].head(3)
    if len(sub) == 0:
        continue
    print(f"\n--- Bloc {b} ---")
    for _, rr in sub.iterrows():
        print(f"Week {rr['Week']} Day {rr['Day']}: pos1={rr['pos1']} | pos2={rr['pos2']} | pos3={rr['pos3']} | pos4={rr['pos4']}"
              + (f"  [Issues: {rr['Issues']}]" if rr['Issues'] else ""))


Blocs détectés: ['Deb', 'Deuxchiffres', 'TroisA', 'TroisB', 'Unchiffe']

Qualité par bloc (résumé):
 - Deb: total=10 ok=10 bad=0
 - Deuxchiffres: total=10 ok=9 bad=1
 - TroisA: total=10 ok=9 bad=1
 - TroisB: total=10 ok=10 bad=0
 - Unchiffe: total=10 ok=10 bad=0

✅ PAGE 2 terminée.
Fichiers produits:
 - ./out/202602/out_pomdp\L0-PAGE2_L1_sequences_by_block_long.csv
 - ./out/202602/out_pomdp\L0-PAGE2_L1_sequences_by_block_list.csv
 - ./out/202602/out_pomdp\L0-PAGE2_L1_sequences_by_block_vector.csv
 - ./out/202602/out_pomdp\L0-PAGE2_block_to_activities.json
 - ./out/202602/out_pomdp\L0-PAGE2_block_quality.json

Aperçu métier (3 premiers jours par bloc) :

--- Bloc Deb ---
Week 1 Day 1: pos1=Lecture du Tableau de nombres | pos2=Activité avec bâtonnets et paquet | pos3=Gymn aux nombres | pos4=Exercices d’opérations
Week 1 Day 2: pos1=Lecture du Tableau de nombres | pos2=Lecture de la Table d’addition | pos3=Opération de base avec problèmes d'addition | pos4=Exercices d’opérations
Week 1 Da

### Section 1 — Mapper automatiquement les “blocs” (préfixes L1) vers les NIVEAUX 1..5 (S = {1..5})

In [ ]:
# -*- coding: utf-8 -*-
"""
Section 1 — Mapper automatiquement les “blocs” (préfixes L1) vers les NIVEAUX 1..5 (S = {1..5})
Objectif 
- Dans L1 (onglet Activités), les colonnes d’activités sont regroupées par blocs (préfixes).
- Dans L3 (features_by_student), tu as déjà une notion de niveau via Pretest_i ∈ {1..5} et (souvent) LevelTag.
- Cette page construit un mapping robuste:
    Block (L1)  ---> Level (1..5)
  en utilisant L3 comme référence (distribution Pretest par LevelTag),
  puis en “matchant” les noms (Block vs LevelTag) par similarité de texte.
"""

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import re
import json
import numpy as np
import pandas as pd
from difflib import SequenceMatcher

# ============================================================
# 0) CONFIG / INPUTS
# ============================================================
OUT_DIR = "./out/202602/out_pomdp"
os.makedirs(OUT_DIR, exist_ok=True)

def out_path(fname: str) -> str:
    return os.path.join(OUT_DIR, f"L0-{fname}")

# Outputs SECTION1
SEQ_LONG = out_path("SECTION2_L1_sequences_by_block_long.csv")
COLBLOCK = out_path("L1_activity_column_blocks.csv")          # from SECTION2
BLOCK_ACTS_JSON = out_path("SECTION2_block_to_activities.json")  # from SECTION2

# L3
L3_PATH = "./data/2025-08/L3.features_by_student.byPretest.csv"  # adapte si besoin
assert os.path.exists(L3_PATH), f"Introuvable L3 : {L3_PATH}"

assert os.path.exists(SEQ_LONG), "Exécute Section 1 d’abord (SECTION2_L1_sequences_by_block_long.csv introuvable)."
assert os.path.exists(COLBLOCK), "Exécute Section 1 d’abord (L1_activity_column_blocks.csv introuvable)."
assert os.path.exists(BLOCK_ACTS_JSON), "Exécute Section 1 d’abord (SECTION2_block_to_activities.json introuvable)."

df_seq_long = pd.read_csv(SEQ_LONG)
df_colblock = pd.read_csv(COLBLOCK)
df_l3 = pd.read_csv(L3_PATH)

# ============================================================
# 1) HELPERS
# ============================================================
def norm_text(s: str) -> str:
    s = str(s).strip().lower()
    s = re.sub(r"[’'`]", " ", s)
    s = re.sub(r"[^a-z0-9]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def sim(a: str, b: str) -> float:
    return SequenceMatcher(None, norm_text(a), norm_text(b)).ratio()

def cap_level(x):
    try:
        v = int(x)
    except Exception:
        return np.nan
    return min(5, max(1, v))

# Assure Pretest_i dans L3
if "Pretest_i" not in df_l3.columns:
    if "Pretest" in df_l3.columns:
        df_l3["Pretest_i"] = df_l3["Pretest"].apply(cap_level)
    else:
        raise ValueError("L3 doit contenir Pretest (ou Pretest_i).")

# ============================================================
# 2) (A) Déduire LevelTag -> Level via distribution Pretest
# ============================================================
leveltag_to_level = {}
leveltag_diag_rows = []

if "LevelTag" in df_l3.columns:
    for tag, g in df_l3.dropna(subset=["LevelTag","Pretest_i"]).groupby("LevelTag"):
        # niveau dominant (mode) dans ce tag
        counts = g["Pretest_i"].astype(int).value_counts().sort_index()
        if len(counts) == 0:
            continue
        dominant_level = int(counts.idxmax())
        share = float(counts.max() / counts.sum())
        leveltag_to_level[str(tag)] = dominant_level
        leveltag_diag_rows.append({
            "LevelTag": str(tag),
            "DominantLevel": dominant_level,
            "DominantShare": round(share, 4),
            "Counts": json.dumps({int(k): int(v) for k,v in counts.to_dict().items()})
        })

df_leveltag_diag = pd.DataFrame(leveltag_diag_rows).sort_values(
    ["DominantLevel","DominantShare"], ascending=[True, False]
)

df_leveltag_diag.to_csv(out_path("SECTION2_leveltag_to_level.csv"), index=False, encoding="utf-8-sig")

print("\n[Diag] LevelTag -> Level (à partir de L3 / Pretest_i):")
if len(df_leveltag_diag) == 0:
    print(" - Aucun LevelTag exploitable trouvé dans L3 (colonne LevelTag absente ou vide).")
else:
    print(df_leveltag_diag.head(30).to_string(index=False))

# ============================================================
# 3) (B) Mapper Block (L1) -> Level via matching texte Block <-> LevelTag
# ============================================================
blocks = sorted(df_colblock["Block"].astype(str).unique().tolist())

# si pas de LevelTag dans L3, fallback: mapping ordinal par fréquence d’apparition
# (moins fiable, mais au moins automatique)
has_leveltag = ("LevelTag" in df_l3.columns) and (len(df_leveltag_diag) > 0)

mapping_rows = []
block_to_level = {}

# Option: override manuel si tu veux forcer certains cas après inspection
MANUAL_OVERRIDE_BLOCK_TO_LEVEL = {
    # Exemple (à compléter si besoin):
    # "TroisA": 4,
    # "TroisB": 5,
}

if has_leveltag:
    tags = list(leveltag_to_level.keys())

    for b in blocks:
        if b in MANUAL_OVERRIDE_BLOCK_TO_LEVEL:
            lvl = int(MANUAL_OVERRIDE_BLOCK_TO_LEVEL[b])
            block_to_level[b] = lvl
            mapping_rows.append({
                "Block": b,
                "MatchedLevelTag": "__MANUAL_OVERRIDE__",
                "Level": lvl,
                "Similarity": 1.0,
                "Method": "manual_override"
            })
            continue

        # meilleur match par similarité texte
        scored = [(t, sim(b, t)) for t in tags]
        scored.sort(key=lambda x: x[1], reverse=True)

        best_tag, best_score = scored[0]
        lvl = int(leveltag_to_level[best_tag])

        # garde aussi les 3 meilleurs pour audit
        top3 = scored[:3]
        mapping_rows.append({
            "Block": b,
            "MatchedLevelTag": best_tag,
            "Level": lvl,
            "Similarity": round(float(best_score), 4),
            "Top3": json.dumps([(t, round(float(sc),4)) for t,sc in top3], ensure_ascii=False),
            "Method": "text_similarity_block_vs_leveltag"
        })
        block_to_level[b] = lvl

else:
    # Fallback: ordonner les blocs par “activité dans le temps”:
    # hypothèse: les blocs “débutants” apparaissent plus tôt (Week/Day petits)
    # => on calcule pour chaque block la médiane du (Week,Day) quand il est réellement utilisé (>=1)
    # puis on mappe les 5 premiers rangs -> niveaux 1..5
    print("\n[WARN] Fallback sans LevelTag: mapping ordinal basé sur la temporalité (moins fiable).")

    # construit un score “timing” par block
    timing = []
    # utilisation d’un jour si au moins une des pos1..pos4 est non nulle
    used = df_seq_long.copy()
    used["is_used"] = used[["pos1","pos2","pos3","pos4"]].notna().any(axis=1).astype(int)

    for b in blocks:
        gb = used[(used["Block"] == b) & (used["is_used"] == 1)]
        if len(gb) == 0:
            # si jamais bloc jamais utilisé (rare), on met un grand score
            med = 10**9
        else:
            # score = median(Week*100 + Day)
            med = float(np.median(gb["Week"].fillna(0).values * 100 + gb["Day"].fillna(0).values))
        timing.append((b, med))

    timing.sort(key=lambda x: x[1])
    # si >5 blocs, on “compresse” sur 1..5 par quantiles
    meds = np.array([t[1] for t in timing], dtype=float)
    if len(meds) == 0:
        raise ValueError("Impossible de construire un mapping (aucun bloc détecté).")

    # quantiles
    q = np.quantile(meds, [0.2, 0.4, 0.6, 0.8])
    def quantile_to_level(m):
        if m <= q[0]: return 1
        if m <= q[1]: return 2
        if m <= q[2]: return 3
        if m <= q[3]: return 4
        return 5

    for b, med in timing:
        if b in MANUAL_OVERRIDE_BLOCK_TO_LEVEL:
            lvl = int(MANUAL_OVERRIDE_BLOCK_TO_LEVEL[b])
            method = "manual_override"
        else:
            lvl = quantile_to_level(med)
            method = "timing_quantile"
        block_to_level[b] = lvl
        mapping_rows.append({
            "Block": b,
            "MatchedLevelTag": "",
            "Level": lvl,
            "Similarity": "",
            "Top3": "",
            "Method": method,
            "MedianWeekDayScore": med
        })

df_map = pd.DataFrame(mapping_rows)

df_map.to_csv(out_path("SECTION2_block_to_level.csv"), index=False, encoding="utf-8-sig")
with open(out_path("SECTION2_block_to_level.json"), "w", encoding="utf-8") as f:
    json.dump(block_to_level, f, ensure_ascii=False, indent=2)

print("\n Mapping Block -> Level produit :")
print(df_map[["Block","Level","MatchedLevelTag","Similarity","Method"]].to_string(index=False))

# ============================================================
# 4) (C) Vérif  : cohérence TroisA/TroisB
# ============================================================
for key in ["TroisA", "TroisB", "troisA", "troisB"]:
    # on normalise par exact match “Block”
    if key in block_to_level:
        print(f"\n[Check] {key} -> Level {block_to_level[key]} (selon mapping)")
        break

print("\nFichiers :")
print(" -", out_path("SECTION2_block_to_level.csv"))
print(" -", out_path("SECTION2_block_to_level.json"))
print(" -", out_path("SECTION2_leveltag_to_level.csv"))



[Diag] LevelTag -> Level (à partir de L3 / Pretest_i):
LevelTag  DominantLevel  DominantShare     Counts
      L1              1            1.0   {"1": 4}
      L2              2            1.0  {"2": 66}
      L3              3            1.0 {"3": 166}
      L4              4            1.0 {"4": 230}
      L5              5            1.0 {"5": 347}

✅ Mapping Block -> Level produit :
       Block  Level MatchedLevelTag  Similarity                            Method
         Deb      1              L1         0.0 text_similarity_block_vs_leveltag
Deuxchiffres      1              L1         0.0 text_similarity_block_vs_leveltag
      TroisA      1              L1         0.0 text_similarity_block_vs_leveltag
      TroisB      1              L1         0.0 text_similarity_block_vs_leveltag
    Unchiffe      1              L1         0.0 text_similarity_block_vs_leveltag

[Check] TroisA -> Level 1 (selon mapping)

Fichiers produits:
 - ./out/202602/out_pomdp\L0-PAGE3_block_to_level.csv

### Section 1 — (1) Définir Action observée A/B via durée réelle (HoursTotal) 
### (2) Apprendre une politique empirique π_ML(a|s,features) (action réellement observée)

In [ ]:
# -*- coding: utf-8 -*-
"""
Section 1 — (1) Définir Action observée A/B via durée réelle (HoursTotal)
         (2) Apprendre une politique empirique π_ML(a|s,features) (action réellement observée)

Rappel  (règles) :
- 1 activité = 2h (fixe)
- 4 activités / jour  => 8h / jour
- Action A = “standard” (≈ 10 jours) => ≈ 10 * 8h = 80h
- Action B = “intensive” (X jours < 10, X inconnu) => HoursTotal < 80h

Donc on étiquette l’action OBSERVÉE par élève (dans L3) via HoursTotal :
- A si DaysTotal >= seuil proche de 10
- B sinon

Puis on entraîne un modèle ML pour apprendre π_ML(a|s,features) :
- Entrées = (state s = Pretest_i) + features (Age/Genre/Zone/LevelTag + freq_pos* + freq_all*)
- Sortie = P(Action=A|x), P(Action=B|x)
"""

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import json
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# optional: xgboost
xgb_ok = True
try:
    import xgboost as xgb
except Exception:
    xgb_ok = False

# persist model
try:
    import joblib
    joblib_ok = True
except Exception:
    joblib_ok = False


# ============================================================
# 0) CONFIG / INPUTS
# ============================================================
OUT_DIR = "./out/202602/out_pomdp"  # adapte si besoin
os.makedirs(OUT_DIR, exist_ok=True)

def out_path(fname: str) -> str:
    return os.path.join(OUT_DIR, f"L0-{fname}")

L3_PATH = "./data/2025-08/L3.features_by_student.byPretest.csv"  # adapte si besoin
assert os.path.exists(L3_PATH), f"Introuvable : {L3_PATH}"

RANDOM_STATE = 42

# 
HOURS_PER_ACTIVITY = 2.0
ACTIVITIES_PER_DAY = 4.0
HOURS_PER_DAY = HOURS_PER_ACTIVITY * ACTIVITIES_PER_DAY   # 8h
STANDARD_DAYS_A = 10.0
STANDARD_HOURS_A = STANDARD_DAYS_A * HOURS_PER_DAY         # 80h

# seuil pour décider A vs B (ajustable)
# -> proche de 10 jours, on tolère un peu d’écart (ex : 9 jours et + => A)
THRESHOLD_DAYS_FOR_A = 9.0

A, B = "A", "B"


# ============================================================
# 1) LOAD L3 + FEATURES
# ============================================================
df = pd.read_csv(L3_PATH)

def cap_level(x):
    try:
        v = int(x)
    except Exception:
        return np.nan
    return min(5, max(1, v))

if "Pretest_i" not in df.columns:
    if "Pretest" in df.columns:
        df["Pretest_i"] = df["Pretest"].apply(cap_level)
    else:
        raise ValueError("L3 doit contenir Pretest ou Pretest_i")

if "HoursTotal" not in df.columns:
    raise ValueError("L3 doit contenir la colonne HoursTotal (tu as demandé de l'utiliser).")


# ============================================================
# 2) LABEL ACTION OBSERVÉE via HoursTotal -> DaysTotal
# ============================================================
df["DaysTotal"] = df["HoursTotal"].astype(float) / float(HOURS_PER_DAY)

# action observée
df["Action_obs"] = np.where(df["DaysTotal"] >= THRESHOLD_DAYS_FOR_A, A, B)

print("\n==================== Section 1 — Action label (A/B) via HoursTotal ====================")
print(f"HOURS_PER_DAY = {HOURS_PER_DAY:.1f}h, STANDARD_HOURS_A≈{STANDARD_HOURS_A:.1f}h (10 jours)")
print(f"Règle de labellisation observée: Action=A si DaysTotal >= {THRESHOLD_DAYS_FOR_A:.1f} jours, sinon B")
print("\nAperçu HoursTotal/DaysTotal/Action_obs:")
print(df[["HoursTotal", "DaysTotal", "Action_obs"]].head(10).to_string(index=False))

print("\nStatistiques DaysTotal:")
print(df["DaysTotal"].describe().to_string())

print("\nRépartition Action_obs:")
print(df["Action_obs"].value_counts(dropna=False).to_string())

df.to_csv(out_path("SECTION1_L3_with_action_labels.csv"), index=False, encoding="utf-8-sig")


# ============================================================
# 3) TEST HYPOTHÈSE (Elysa) sur OBSERVÉ: niveaux 1-2 => A, niveaux 3-4-5 => B
# ============================================================
def summarize_action_by_level(df0: pd.DataFrame, label_col="Action_obs"):
    rows = []
    tmp = df0.dropna(subset=["Pretest_i"]).copy()
    for lvl, g in tmp.groupby("Pretest_i"):
        lvl = int(lvl)
        n = len(g)
        pA = float((g[label_col] == A).mean()) if n > 0 else np.nan
        pB = float((g[label_col] == B).mean()) if n > 0 else np.nan
        rows.append({"Level": lvl, "N": n, "P(A)": round(pA, 4), "P(B)": round(pB, 4)})
    return pd.DataFrame(rows).sort_values("Level")

df_hyp_obs = summarize_action_by_level(df, label_col="Action_obs")
print("\n[Hypothèse Elysa] Répartition observée (Action_obs) par niveau Pretest:")
print(df_hyp_obs.to_string(index=False))
df_hyp_obs.to_csv(out_path("SECTION2_hypothesis_obs_by_level.csv"), index=False, encoding="utf-8-sig")


# ============================================================
# 4) APPRENDRE π_ML(a|s,features)  (classification binaire A vs B)
# ============================================================
demographic_cols = [c for c in ["Age", "Genre", "Zone", "LevelTag"] if c in df.columns]
state_col = "Pretest_i"

freq_cols = [c for c in df.columns if c.startswith("freq_pos") or c.startswith("freq_all")]
feature_cols = [state_col] + demographic_cols + freq_cols

missing = [c for c in feature_cols if c not in df.columns]
if missing:
    raise ValueError(f"Colonnes manquantes dans L3 pour apprendre π_ML : {missing}")

X = df[feature_cols].copy()
y = (df["Action_obs"] == A).astype(int)  # 1=A, 0=B

print("\n[DIAG] Répartition y (Action_obs==A):")
print(y.value_counts(dropna=False).to_string())
print("PosRate(y=1) =", float(y.mean()))

# preprocessing
num_cols, cat_cols = [], []
for c in X.columns:
    if c in ["Genre", "Zone", "LevelTag"]:
        cat_cols.append(c)
    else:
        num_cols.append(c)

preproc = ColumnTransformer([
    ("num", Pipeline([("imp", SimpleImputer(strategy="median")),
                      ("sc", StandardScaler())]),
     num_cols),
    ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                      ("oh", OneHotEncoder(handle_unknown="ignore"))]),
     cat_cols)
], remainder="drop")

# candidats (avec class_weight pour stabilité quand déséquilibré)
candidates = {
    "RandomForestClassifier": Pipeline([("prep", preproc),
                                        ("m", RandomForestClassifier(
                                            n_estimators=250, random_state=RANDOM_STATE,
                                            max_depth=12, n_jobs=1,
                                            class_weight="balanced"
                                        ))]),
    "GradientBoostingClassifier": Pipeline([("prep", preproc),
                                           ("m", GradientBoostingClassifier(
                                               random_state=RANDOM_STATE, n_estimators=250
                                           ))]),
    "LogisticRegression": Pipeline([("prep", preproc),
                                    ("m", LogisticRegression(
                                        max_iter=5000,
                                        class_weight="balanced"
                                    ))]),
    "SVC": Pipeline([("prep", preproc),
                     ("m", SVC(
                         probability=True,
                         random_state=RANDOM_STATE,
                         class_weight="balanced"
                     ))]),
    ("XGBoostClassifier" if xgb_ok else "HistGradientBoostingClassifier"): Pipeline([
        ("prep", preproc),
        ("m", xgb.XGBClassifier(
            n_estimators=300, learning_rate=0.07, max_depth=5, subsample=0.9,
            colsample_bytree=0.9, random_state=RANDOM_STATE, n_jobs=1,
            eval_metric="logloss"
        ) if xgb_ok else HistGradientBoostingClassifier(random_state=RANDOM_STATE))
    ])
}

# ---------- Robust proba helpers ----------
def _sigmoid(z):
    z = np.asarray(z, dtype=float)
    return 1.0 / (1.0 + np.exp(-z))

def _proba_class1(fitted_pipe, X_batch):
    """
    Retourne P(y=1|x) robuste :
    - si predict_proba dispo et a 2 colonnes => ok (en respectant classes_)
    - si une seule classe vue à l'entraînement => renvoie 0 ou 1 constant selon la classe
    - sinon fallback decision_function -> sigmoid
    """
    m = fitted_pipe

    if hasattr(m, "predict_proba"):
        proba = m.predict_proba(X_batch)

        # cas normal: 2 colonnes
        if proba.shape[1] >= 2:
            cls = getattr(m, "classes_", None)
            if cls is None:
                return proba[:, 1]
            cls = list(cls)
            if 1 in cls:
                return proba[:, cls.index(1)]
            return proba[:, 1]

        # cas 1 colonne => une seule classe dans ce fold
        cls = getattr(m, "classes_", None)
        if cls is None:
            return np.zeros(len(X_batch), dtype=float)
        only = int(list(cls)[0])
        return np.full(len(X_batch), float(only), dtype=float)

    if hasattr(m, "decision_function"):
        scores = m.decision_function(X_batch)
        scores = np.asarray(scores).ravel()
        return _sigmoid(scores)

    # dernier fallback
    pred = m.predict(X_batch)
    return np.asarray(pred, dtype=float)


def eval_policy_ml_cv(X, y, pipe, n_splits=5):
    """
    Évaluation CV robuste pour π_ML.
    - StratifiedKFold pour préserver le ratio A/B.
    - Gère les folds à une seule classe (sinon predict_proba[:,1] plante).
    """
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)
    probs_all, preds_all, y_all = [], [], []

    for tr, te in skf.split(X, y):
        m = clone(pipe)
        y_tr = y.iloc[tr]

        # Si le fold train contient une seule classe => proba constante
        if len(np.unique(y_tr)) < 2:
            only = int(np.unique(y_tr)[0])
            pr = np.full(len(te), float(only), dtype=float)
        else:
            m.fit(X.iloc[tr], y_tr)
            pr = _proba_class1(m, X.iloc[te])

        pd_ = (pr >= 0.5).astype(int)

        probs_all.extend(pr.tolist())
        preds_all.extend(pd_.tolist())
        y_all.extend(y.iloc[te].tolist())

    probs_all = np.array(probs_all, dtype=float)
    preds_all = np.array(preds_all, dtype=int)
    y_all = np.array(y_all, dtype=int)

    auc = np.nan
    if len(np.unique(y_all)) > 1:
        auc = float(roc_auc_score(y_all, probs_all))

    return {
        "AUC": auc,
        "F1": float(f1_score(y_all, preds_all)),
        "ACC": float(accuracy_score(y_all, preds_all)),
        "PosRate(y=1)": float(y.mean()),
        "PredPosRate": float(preds_all.mean())
    }


print("\n==================== Section 2 — Model selection for π_ML(a|s,features) ====================")
rows = []
scores = {}

for name, pipe in candidates.items():
    sc = eval_policy_ml_cv(X, y, pipe, n_splits=5)
    scores[name] = sc
    rows.append({
        "Model": name,
        "AUC": (round(sc["AUC"], 4) if isinstance(sc["AUC"], float) and not np.isnan(sc["AUC"]) else sc["AUC"]),
        "F1": round(sc["F1"], 4),
        "ACC": round(sc["ACC"], 4),
        "PosRate(y=1)": round(sc["PosRate(y=1)"], 4),
        "PredPosRate": round(sc["PredPosRate"], 4),
    })

df_sel = pd.DataFrame(rows)

# tri robuste : AUC d'abord (NaN -> -1), puis F1 puis ACC
def _auc_for_sort(v):
    try:
        if v is None or (isinstance(v, float) and np.isnan(v)):
            return -1.0
        return float(v)
    except Exception:
        return -1.0

df_sel["_AUCsort"] = df_sel["AUC"].apply(_auc_for_sort)
df_sel = df_sel.sort_values(["_AUCsort", "F1", "ACC"], ascending=False).drop(columns=["_AUCsort"])

print(df_sel.to_string(index=False))
df_sel.to_csv(out_path("SECTION2_policyML_model_selection.csv"), index=False, encoding="utf-8-sig")

# choisir le meilleur
best_model_name = df_sel.iloc[0]["Model"]
best_pipe = candidates[best_model_name]
best_pipe.fit(X, y)
print(f"\n Best π_ML model = {best_model_name}")

# persister (si joblib dispo)
if joblib_ok:
    joblib.dump(best_pipe, out_path("SECTION2_best_policyML_model.joblib"))
    with open(out_path("SECTION2_best_policyML_model_meta.json"), "w", encoding="utf-8") as f:
        json.dump({
            "best_model_name": best_model_name,
            "threshold_days_for_A": THRESHOLD_DAYS_FOR_A,
            "hours_per_day": HOURS_PER_DAY,
            "standard_days_A": STANDARD_DAYS_A,
            "standard_hours_A": STANDARD_HOURS_A
        }, f, ensure_ascii=False, indent=2)
else:
    print("[WARN] joblib indisponible: le modèle n'est pas sérialisé.")


# ============================================================
# 5) PRODUITS: π_ML prédictions + analyse hypothèse sur π_ML
# ============================================================
# prédire P(A|x) de façon robuste (même si classes_ bizarre)
pA = _proba_class1(best_pipe, X)

df["piML_P(A)"] = pA
df["piML_Action_hat"] = np.where(df["piML_P(A)"] >= 0.5, A, B)

pred_cols = ["Pretest_i", "HoursTotal", "DaysTotal", "Action_obs", "piML_P(A)", "piML_Action_hat"]
for c in ["Age", "Genre", "Zone", "LevelTag"]:
    if c in df.columns:
        pred_cols.insert(1, c)

df[pred_cols].to_csv(out_path("SECTION2_policyML_predictions.csv"), index=False, encoding="utf-8-sig")

# hypothèse Elysa sur π_ML
df_hyp_ml = summarize_action_by_level(df, label_col="piML_Action_hat")
print("\n[Hypothèse Elysa] Décision π_ML (Action_hat) par niveau Pretest:")
print(df_hyp_ml.to_string(index=False))
df_hyp_ml.to_csv(out_path("SECTION2_hypothesis_piML_by_level.csv"), index=False, encoding="utf-8-sig")

# synthèse groupée “support hypothèse”
support_rows = []
for group_name, levels in [("Low (1-2)", [1, 2]), ("High (3-5)", [3, 4, 5])]:
    g = df[df["Pretest_i"].isin(levels)]
    if len(g) == 0:
        continue
    pA_obs = float((g["Action_obs"] == A).mean())
    pA_ml  = float((g["piML_Action_hat"] == A).mean())
    support_rows.append({
        "Group": group_name,
        "N": int(len(g)),
        "Obs P(A)": round(pA_obs, 4),
        "π_ML P(A_hat)": round(pA_ml, 4),
        "Expected": ("High" if group_name.startswith("Low") else "Low")
    })

df_support = pd.DataFrame(support_rows)
print("\n[Hypothèse Elysa] Synthèse groupée:")
print(df_support.to_string(index=False))
df_support.to_csv(out_path("SECTION2_hypothesis_support_summary.csv"), index=False, encoding="utf-8-sig")

print("\nFichiers :")
print(" -", out_path("SECTION2_L3_with_action_labels.csv"))
print(" -", out_path("SECTION2_policyML_model_selection.csv"))
print(" -", out_path("SECTION2_policyML_predictions.csv"))
print(" -", out_path("SECTION2_hypothesis_obs_by_level.csv"))
print(" -", out_path("SECTION2_hypothesis_piML_by_level.csv"))
print(" -", out_path("SECTION1_hypothesis_support_summary.csv"))
if joblib_ok:
    print(" -", out_path("SECTION2_best_policyML_model.joblib"))
    print(" -", out_path("SECTION2_best_policyML_model_meta.json"))

print("\nXGBoost disponible ?", xgb_ok)



==================== PAGE 4 — Action label (A/B) via HoursTotal ====================
HOURS_PER_DAY = 8.0h, STANDARD_HOURS_A≈80.0h (10 jours)
Règle de labellisation observée: Action=A si DaysTotal >= 9.0 jours, sinon B

Aperçu HoursTotal/DaysTotal/Action_obs:
 HoursTotal  DaysTotal Action_obs
         60        7.5          B
         60        7.5          B
         60        7.5          B
         60        7.5          B
         60        7.5          B
         60        7.5          B
         60        7.5          B
         60        7.5          B
         60        7.5          B
         60        7.5          B

Statistiques DaysTotal:
count    813.000000
mean       5.648831
std        2.116959
min        2.500000
25%        2.500000
50%        7.500000
75%        7.500000
max        7.500000

Répartition Action_obs:
Action_obs
B    813

[Hypothèse Elysa] Répartition observée (Action_obs) par niveau Pretest:
 Level   N  P(A)  P(B)
     1   4   0.0   1.0
     2  66   0.0 

### Section 1 — Optimisation du temps réel (heures / jours) pour atteindre l’état absorbant (5)

In [ ]:
# -*- coding: utf-8 -*-
"""
Section 1 — Optimisation du temps réel (heures / jours) pour atteindre l’état absorbant (5)

Objectifs :
- Calculer le temps attendu (heures/jours) pour atteindre l’état 5
  sous :
  - π_A (A partout)
  - π*_hours (Bellman en heures)
  - π_ML_state (politique apprise par ML, agrégée par niveau)
"""

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os, json
import numpy as np
import pandas as pd

EPS = 1e-12
A, B = "A", "B"
LEVELS = [1, 2, 3, 4, 5]
TRANSIENT = [1, 2, 3, 4]
ABSORBING = 5
N_LEVELS = 5

# ------------------------------------------------------------
# CONFIG TEMPS (règles )
# ------------------------------------------------------------
HOURS_PER_ACTIVITY = 2.0
ACTIVITIES_PER_DAY = 4.0
HOURS_PER_DAY = HOURS_PER_ACTIVITY * ACTIVITIES_PER_DAY   # 8h/jour

# ------------------------------------------------------------
# PATHS
# ------------------------------------------------------------
OUT_DIR = "./out/202602/out_pomdp"   # adapte si besoin
os.makedirs(OUT_DIR, exist_ok=True)

def out_path(fname: str) -> str:
    return os.path.join(OUT_DIR, f"L5-{fname}")

L3_PATH = "./data/2025-08/L3.features_by_student.byPretest.csv"  # adapte si besoin
assert os.path.exists(L3_PATH), f"Introuvable : {L3_PATH}"

# Si tu as déjà exporté T_A/T_B dans ton code principal, tu peux les charger ici
TA_CSV = os.path.join(OUT_DIR, "L4-AMC_TA_standard.csv")   # from principal
TB_CSV = os.path.join(OUT_DIR, "L4-AMC_TB_intensive.csv")  # from principal

# ------------------------------------------------------------
# UTILITAIRES
# ------------------------------------------------------------
def cap_level(x):
    try:
        v = int(x)
    except Exception:
        return np.nan
    return min(5, max(1, v))

def canonical_QR(T: np.ndarray):
    Q = T[:len(TRANSIENT), :len(TRANSIENT)]
    R = T[:len(TRANSIENT), len(TRANSIENT):]
    return Q, R

def fundamental_matrix_N(Q: np.ndarray):
    I = np.eye(Q.shape[0], dtype=float)
    M = I - Q
    try:
        return np.linalg.inv(M)
    except np.linalg.LinAlgError:
        return np.linalg.pinv(M)

def expected_time_with_costs(T: np.ndarray, cost_vec_transient: np.ndarray):
    """
    Expected cumulative cost until absorption:
      m_cost = N * c
    cost_vec_transient: c_i (heures) pour i=1..4
    """
    Q, _ = canonical_QR(T)
    N = fundamental_matrix_N(Q)
    m = N.dot(cost_vec_transient.reshape(-1, 1)).flatten()
    return {"Q": Q, "N": N, "m": m, "J": float(m.sum())}

def build_T_pi(policy: dict, TA: np.ndarray, TB: np.ndarray) -> np.ndarray:
    T = np.zeros((N_LEVELS, N_LEVELS), dtype=float)
    for s in LEVELS:
        if s == ABSORBING:
            T[s-1, :] = 0.0
            T[s-1, ABSORBING-1] = 1.0
        else:
            act = policy.get(s, A)
            T[s-1, :] = TA[s-1, :] if act == A else TB[s-1, :]
    return T

def policy_all_A():
    return {s: A for s in TRANSIENT}

def bellman_value_iteration_time_cost(TA: np.ndarray, TB: np.ndarray, cost_hours: np.ndarray,
                                      max_iter=20000, tol=1e-12):
    """
    Bellman en HEURES :
      V(5)=0
      V(s)=c(s) + min_a sum_{s'} P(s'|s,a) V(s')
    cost_hours: length 5, cost_hours[4]=0
    """
    V = np.zeros(N_LEVELS, dtype=float)
    V[:4] = float(np.max(cost_hours[:4])) * 5.0
    pol = {s: A for s in TRANSIENT}

    for _ in range(max_iter):
        V_old = V.copy()
        for s in TRANSIENT:
            c = float(cost_hours[s-1])
            qA = c + float(np.dot(TA[s-1, :], V_old))
            qB = c + float(np.dot(TB[s-1, :], V_old))
            if qB < qA:
                V[s-1] = qB
                pol[s] = B
            else:
                V[s-1] = qA
                pol[s] = A
        V[ABSORBING-1] = 0.0
        if np.max(np.abs(V - V_old)) < tol:
            break
    return V, pol

def policy_from_piML_predictions(df_with_piML: pd.DataFrame,
                                level_col="Pretest_i",
                                probaA_col="piML_P(A)",
                                threshold=0.5):
    """
    π_ML_state(s) = A si mean(P(A|x)) >= threshold sur les élèves du niveau s, sinon B.
    """
    pol = {}
    for s in TRANSIENT:
        g = df_with_piML[df_with_piML[level_col] == s]
        if len(g) == 0:
            pol[s] = A
            continue
        pA = float(g[probaA_col].mean())
        pol[s] = A if pA >= threshold else B
    return pol

# ------------------------------------------------------------
# (0) LOAD DF
# ------------------------------------------------------------
df = pd.read_csv(L3_PATH)

if "Pretest_i" not in df.columns:
    if "Pretest" in df.columns:
        df["Pretest_i"] = df["Pretest"].apply(cap_level)
    else:
        raise ValueError("L3 doit contenir Pretest ou Pretest_i")

if "HoursTotal" not in df.columns:
    raise ValueError("L3 doit contenir HoursTotal")

# ------------------------------------------------------------
# (1) DEFINIR / CHARGER T_A, T_B
# ------------------------------------------------------------
def empirical_transition_matrix_from_pretest_final(df: pd.DataFrame) -> np.ndarray:
    """
    Estimation empirique de P(Final=j | Pretest=i) (Action A — Standard).
    """
    if "Final_i" not in df.columns:
        if "Final" in df.columns:
            df = df.copy()
            df["Final_i"] = df["Final"].apply(cap_level)
        else:
            raise ValueError("L3 doit contenir Final ou Final_i pour estimer T_A empirique.")

    mat_counts = np.zeros((N_LEVELS, N_LEVELS), dtype=float)
    dfx = df.dropna(subset=["Pretest_i", "Final_i"]).copy()

    for _, r in dfx.iterrows():
        i = int(r["Pretest_i"]) - 1
        j = int(r["Final_i"]) - 1
        mat_counts[i, j] += 1.0

    T = np.zeros_like(mat_counts)
    for i in range(N_LEVELS):
        s = mat_counts[i, :].sum()
        if s > 0:
            T[i, :] = mat_counts[i, :] / s
        else:
            T[i, :] = 1.0 / N_LEVELS

    # force absorbing state 5
    T[ABSORBING-1, :] = 0.0
    T[ABSORBING-1, ABSORBING-1] = 1.0
    return T

def make_intensive_matrix_from_standard(TA: np.ndarray,
                                       diag_shrink: float = 0.80,
                                       regress_shrink: float = 0.70,
                                       boost_to_absorb: float = 1.25) -> np.ndarray:
    """
    Action B — Intensive : contre-factuel normatif (non causal)
    """
    TB = TA.copy().astype(float)
    for i in range(N_LEVELS):
        if i == ABSORBING-1:
            continue
        for j in range(N_LEVELS):
            if j == i:
                TB[i, j] *= diag_shrink
            elif j < i:
                TB[i, j] *= regress_shrink
        TB[i, ABSORBING-1] *= boost_to_absorb
        s = TB[i, :].sum()
        TB[i, :] = (TB[i, :] / s) if s > 0 else (1.0 / N_LEVELS)

    TB[ABSORBING-1, :] = 0.0
    TB[ABSORBING-1, ABSORBING-1] = 1.0
    return TB

def load_T_from_csv(path: str) -> np.ndarray:
    T = pd.read_csv(path, index_col=0)
    # assure shape 5x5
    arr = T.values.astype(float)
    if arr.shape != (5, 5):
        raise ValueError(f"Matrice {path} doit être 5x5, reçu {arr.shape}")
    return arr

# Priorité: charger depuis CSV L4 si dispo, sinon reconstruire depuis df
if os.path.exists(TA_CSV) and os.path.exists(TB_CSV):
    T_A = load_T_from_csv(TA_CSV)
    T_B = load_T_from_csv(TB_CSV)
    print("\n[INFO] T_A/T_B chargées depuis CSV L4:", TA_CSV, TB_CSV)
else:
    print("\n[INFO] CSV L4 introuvables, reconstruction de T_A/T_B depuis df (L3).")
    T_A = empirical_transition_matrix_from_pretest_final(df)
    T_B = make_intensive_matrix_from_standard(T_A)

# ------------------------------------------------------------
# (2) CALIBRER COÛTS EN HEURES PAR ÉTAT (option 1 fixe, option 2 data)
# ------------------------------------------------------------
cost_hours_fixed = np.array([HOURS_PER_DAY, HOURS_PER_DAY, HOURS_PER_DAY, HOURS_PER_DAY, 0.0], dtype=float)

def calibrate_hours_total_by_level(df: pd.DataFrame):
    stats = (
        df.dropna(subset=["Pretest_i", "HoursTotal"])
          .groupby("Pretest_i")["HoursTotal"]
          .agg(["count", "mean", "median"])
          .reset_index()
          .rename(columns={"Pretest_i": "Level", "count": "N", "mean": "HoursMean", "median": "HoursMedian"})
    )
    return stats.sort_values("Level")

def derive_cost_hours_by_state_from_data(df: pd.DataFrame, T_under_policyA: np.ndarray):
    """
    c_i ≈ mean(HoursTotal | Pretest=i) / t_i(π_A)  (heures par 'étape')
    où t_i(π_A) vient de N=(I-Q)^-1 sous π_A, en unités “étapes”.
    """
    Q, _ = canonical_QR(T_under_policyA)
    N = fundamental_matrix_N(Q)
    t_steps = N.sum(axis=1)  # length 4

    stats = calibrate_hours_total_by_level(df)
    c = np.array([HOURS_PER_DAY]*4, dtype=float)

    for idx, s in enumerate(TRANSIENT):
        row = stats[stats["Level"] == s]
        if len(row) == 0:
            continue
        mean_hours = float(row.iloc[0]["HoursMean"])
        denom = float(t_steps[idx])
        if denom > EPS and mean_hours > EPS:
            c[idx] = mean_hours / denom

    c = np.clip(c, 2.0, 12.0)  # garde-fou
    return np.array([c[0], c[1], c[2], c[3], 0.0], dtype=float), stats, t_steps

# Policy π_A
piA = policy_all_A()
T_piA = build_T_pi(piA, T_A, T_B)

cost_hours_calib, stats_hours, t_steps_piA = derive_cost_hours_by_state_from_data(df, T_piA)

# ------------------------------------------------------------
# (3) CALCULS : π_A, π*_hours, π_ML_state
# ------------------------------------------------------------
print("\n==================== Section 1 — Expected time-to-absorption (hours/days) ====================")
print(f"HOURS_PER_DAY = {HOURS_PER_DAY:.1f}h (4 activités * 2h)")
print("\n--- Coûts c(s) en heures (par étape) ---")
print("Option FIXE ():", {s: float(cost_hours_fixed[s-1]) for s in LEVELS})
print("Option CALIBRÉE (data):", {s: float(cost_hours_calib[s-1]) for s in LEVELS})

print("\nHeures observées (HoursTotal) par niveau initial (L3):")
print(stats_hours.to_string(index=False))

res_piA_fixed = expected_time_with_costs(T_piA, cost_hours_fixed[:4])
res_piA_calib = expected_time_with_costs(T_piA, cost_hours_calib[:4])

def _print_expected(res, label, cost_hours_vec):
    print(f"\n[{label}] Temps attendu jusqu’à maîtrise (état 5) — par niveau initial")
    rows = []
    for i, s in enumerate(TRANSIENT):
        hours = float(res["m"][i])
        days = hours / HOURS_PER_DAY
        rows.append({
            "StartLevel": s,
            "ExpectedHours": round(hours, 2),
            "ExpectedDays": round(days, 2),
            "CostHoursPerStep": round(float(cost_hours_vec[s-1]), 2)
        })
    df_out = pd.DataFrame(rows)
    print(df_out.to_string(index=False))
    return df_out

df_piA_fixed = _print_expected(res_piA_fixed, "π_A (A partout) + coût FIXE", cost_hours_fixed)
df_piA_calib = _print_expected(res_piA_calib, "π_A (A partout) + coût CALIBRÉ", cost_hours_calib)

# Bellman optimal en HEURES => π*_hours
V_hours_star, pi_hours_star = bellman_value_iteration_time_cost(T_A, T_B, cost_hours_calib)
T_pi_hours_star = build_T_pi(pi_hours_star, T_A, T_B)
res_pi_star_hours = expected_time_with_costs(T_pi_hours_star, cost_hours_calib[:4])

print("\n--- Politique optimale π*_hours (Bellman en heures) ---")
print("π*_hours (state->A/B):", pi_hours_star)
print("V*_hours (états 1..5):", [round(float(x), 3) for x in V_hours_star])

df_pi_star_hours = _print_expected(res_pi_star_hours, "π*_hours (optimal temps) + coût CALIBRÉ", cost_hours_calib)

# π_ML_state : agrégée par niveau si Section 1 a été exécutée et a produit piML_P(A)
pi_ml_state, df_pi_ml = None, None
if "piML_P(A)" in df.columns:
    pi_ml_state = policy_from_piML_predictions(df, level_col="Pretest_i", probaA_col="piML_P(A)", threshold=0.5)
    T_pi_ml_state = build_T_pi(pi_ml_state, T_A, T_B)
    res_pi_ml = expected_time_with_costs(T_pi_ml_state, cost_hours_calib[:4])

    print("\n--- Politique π_ML_state (agrégée par niveau) ---")
    print("π_ML_state (state->A/B):", pi_ml_state)
    df_pi_ml = _print_expected(res_pi_ml, "π_ML_state + coût CALIBRÉ", cost_hours_calib)
else:
    print("\n[INFO] df ne contient pas piML_P(A). Exécute Section 1 avant si tu veux π_ML_state.")

# ------------------------------------------------------------
# (4) EXPORTS
# ------------------------------------------------------------
export = {
    "HOURS_PER_DAY": HOURS_PER_DAY,
    "cost_hours_fixed": cost_hours_fixed.tolist(),
    "cost_hours_calibrated": cost_hours_calib.tolist(),
    "piA": piA,
    "pi_hours_star": pi_hours_star,
    "V_hours_star": V_hours_star.tolist(),
    "pi_ml_state": (pi_ml_state if pi_ml_state is not None else None),
    "expected_piA_fixed_hours": df_piA_fixed.to_dict(orient="records"),
    "expected_piA_calib_hours": df_piA_calib.to_dict(orient="records"),
    "expected_pi_star_hours": df_pi_star_hours.to_dict(orient="records"),
    "expected_pi_ml_hours": (df_pi_ml.to_dict(orient="records") if df_pi_ml is not None else None)
}

with open(out_path("expected_time_to_absorption_hours_days.json"), "w", encoding="utf-8") as f:
    json.dump(export, f, ensure_ascii=False, indent=2)

pd.DataFrame(export["expected_piA_fixed_hours"]).to_csv(out_path("expected_piA_fixed.csv"), index=False, encoding="utf-8-sig")
pd.DataFrame(export["expected_piA_calib_hours"]).to_csv(out_path("expected_piA_calib.csv"), index=False, encoding="utf-8-sig")
pd.DataFrame(export["expected_pi_star_hours"]).to_csv(out_path("expected_pi_star_hours.csv"), index=False, encoding="utf-8-sig")
if export["expected_pi_ml_hours"] is not None:
    pd.DataFrame(export["expected_pi_ml_hours"]).to_csv(out_path("expected_pi_ml_hours.csv"), index=False, encoding="utf-8-sig")

print("\n Exports (Section 1):")
print(" -", out_path("expected_time_to_absorption_hours_days.json"))
print(" -", out_path("expected_piA_fixed.csv"))
print(" -", out_path("expected_piA_calib.csv"))
print(" -", out_path("expected_pi_star_hours.csv"))
if export["expected_pi_ml_hours"] is not None:
    print(" -", out_path("expected_pi_ml_hours.csv"))

print("\n==================== INTERPRÉTATION  ====================")
print("1) ExpectedHours = temps moyen (en heures) attendu pour atteindre le niveau 5 (maîtrise),")
print("   en partant du niveau initial, sous une politique donnée (π_A, π*_hours, π_ML_state).")
print("2) ExpectedDays = ExpectedHours / 8h, car 4 activités/jour et 2h/activité => 8h/jour.")
print("3) π*_hours est la politique optimale trouvée par Bellman quand on minimise le TEMPS (heures) et non le nombre d’étapes.")
print("4) La version 'coût calibré' utilise HoursTotal observé pour approximer un coût par niveau (heures/étape).")
print("5) Si π*_hours réduit ExpectedHours vs π_A pour certains niveaux, alors l’action B (intensive) est utile sur ces niveaux.")



[INFO] CSV L4 introuvables, reconstruction de T_A/T_B depuis df (L3).

==================== PAGE 5 — Expected time-to-absorption (hours/days) ====================
HOURS_PER_DAY = 8.0h (4 activités * 2h)

--- Coûts c(s) en heures (par étape) ---
Option FIXE (métier): {1: 8.0, 2: 8.0, 3: 8.0, 4: 8.0, 5: 0.0}
Option CALIBRÉE (data): {1: 12.0, 2: 12.0, 3: 12.0, 4: 12.0, 5: 0.0}

Heures observées (HoursTotal) par niveau initial (L3):
 Level   N  HoursMean  HoursMedian
     1   4  55.000000         60.0
     2  66  50.909091         60.0
     3 166  51.445783         60.0
     4 230  35.565217         20.0
     5 347  47.377522         60.0

[π_A (A partout) + coût FIXE] Temps attendu jusqu’à maîtrise (état 5) — par niveau initial
 StartLevel  ExpectedHours  ExpectedDays  CostHoursPerStep
          1          29.88          3.73               8.0
          2          27.76          3.47               8.0
          3          19.34          2.42               8.0
          4          19.53  

### Section 1 — Planification “jour par jour” avec durée VARIABLE (X jours) jusqu’à absorption (niveau 5)

In [ ]:
# -*- coding: utf-8 -*-
"""
Section 1 — Planification “jour par jour” avec durée VARIABLE (X jours) jusqu’à absorption (niveau 5)

But  :
- Ne plus forcer "10 jours".
- Déterminer automatiquement X (nombre de jours) nécessaires pour atteindre l’état absorbant 5,
  sous une politique π (π*_hours optimisée en temps, ou π_ML_state).

Sorties :
- Plans MDP auto-stop (most_likely + sample)
- Plan POMDP auto-stop si les variables HMM/POMDP existent déjà dans le namespace
- Export JSON + CSV + TXT dans OUT_DIR

Pré-requis  (doivent exister quelque part dans ton code/projet) :
- activities_for_level(level)        -> retourne la liste des activités d’un niveau
- action_to_sequence(level, action)  -> retourne {pos1..pos4} pour (niveau, action A/B)
"""

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os, json
import numpy as np
import pandas as pd

EPS = 1e-12
A, B = "A", "B"
LEVELS = [1, 2, 3, 4, 5]
TRANSIENT = [1, 2, 3, 4]
ABSORBING = 5
N_LEVELS = 5

# Durée max de sécurité (évite boucle infinie si transitions ne mènent pas à 5)
MAX_DAYS_CAP = 60

# ------------------------------------------------------------
# CONFIG TEMPS (règles )
# ------------------------------------------------------------
HOURS_PER_ACTIVITY = 2.0
ACTIVITIES_PER_DAY = 4.0
HOURS_PER_DAY = HOURS_PER_ACTIVITY * ACTIVITIES_PER_DAY  # 8h/jour

# ------------------------------------------------------------
# PATHS ( demandé : utiliser ./out/202602/out_pomdp)
# ------------------------------------------------------------
OUT_DIR = "./out/202602/out_pomdp"
os.makedirs(OUT_DIR, exist_ok=True)

def out_path(fname: str) -> str:
    return os.path.join(OUT_DIR, f"L6-{fname}")

# Données
L3_PATH = "./data/2025-08/L3.features_by_student.byPretest.csv"  # adapte si besoin
assert os.path.exists(L3_PATH), f"Introuvable : {L3_PATH}"

# Matrices L4 (si ton code principal les a déjà exportées)
TA_CSV = os.path.join(OUT_DIR, "L4-AMC_TA_standard.csv")
TB_CSV = os.path.join(OUT_DIR, "L4-AMC_TB_intensive.csv")

# Résultats Section 1 (si disponibles)
SECTION1_JSON = os.path.join(OUT_DIR, "L5-expected_time_to_absorption_hours_days.json")

# ------------------------------------------------------------
# DEFAULTS si variables non définies dans ton notebook
# ------------------------------------------------------------
#  Correction de ton erreur : START_LEVEL absent
START_LEVEL = int(globals().get("START_LEVEL", 1))
TARGET_LEVEL = int(globals().get("TARGET_LEVEL", 5))

BY_MODEL_NAME = globals().get("BY_MODEL_NAME", "PolicyTimeOptimal")
TECHNIQUE_MDP = globals().get("TECHNIQUE_MDP", "Baseline MDP (fully observed)")
TECHNIQUE_POMDP = globals().get("TECHNIQUE_POMDP", "POMDP approx (HMM belief + projection)")

# ------------------------------------------------------------
# OUTILS GÉNÉRIQUES
# ------------------------------------------------------------
def cap_level(x):
    try:
        v = int(x)
    except Exception:
        return np.nan
    return min(5, max(1, v))

def _safe_tag(s: str) -> str:
    return "".join(ch if ch.isalnum() or ch in "-_." else "_" for ch in str(s))

def load_T_from_csv(path: str) -> np.ndarray:
    T = pd.read_csv(path, index_col=0)
    arr = T.values.astype(float)
    if arr.shape != (5, 5):
        raise ValueError(f"Matrice {path} doit être 5x5, reçu {arr.shape}")
    return arr

def empirical_transition_matrix_from_pretest_final(df: pd.DataFrame) -> np.ndarray:
    """
    Estimation empirique de P(Final=j | Pretest=i) (Action A — Standard).
    """
    if "Final_i" not in df.columns:
        if "Final" in df.columns:
            df = df.copy()
            df["Final_i"] = df["Final"].apply(cap_level)
        else:
            raise ValueError("L3 doit contenir Final ou Final_i pour estimer T_A empirique.")

    mat_counts = np.zeros((N_LEVELS, N_LEVELS), dtype=float)
    dfx = df.dropna(subset=["Pretest_i", "Final_i"]).copy()

    for _, r in dfx.iterrows():
        i = int(r["Pretest_i"]) - 1
        j = int(r["Final_i"]) - 1
        mat_counts[i, j] += 1.0

    T = np.zeros_like(mat_counts)
    for i in range(N_LEVELS):
        s = mat_counts[i, :].sum()
        if s > 0:
            T[i, :] = mat_counts[i, :] / s
        else:
            T[i, :] = 1.0 / N_LEVELS

    T[ABSORBING-1, :] = 0.0
    T[ABSORBING-1, ABSORBING-1] = 1.0
    return T

def make_intensive_matrix_from_standard(TA: np.ndarray,
                                       diag_shrink: float = 0.80,
                                       regress_shrink: float = 0.70,
                                       boost_to_absorb: float = 1.25) -> np.ndarray:
    """
    Action B — Intensive : contre-factuel normatif (non causal).
    """
    TB = TA.copy().astype(float)
    for i in range(N_LEVELS):
        if i == ABSORBING-1:
            continue
        for j in range(N_LEVELS):
            if j == i:
                TB[i, j] *= diag_shrink
            elif j < i:
                TB[i, j] *= regress_shrink
        TB[i, ABSORBING-1] *= boost_to_absorb
        s = TB[i, :].sum()
        TB[i, :] = (TB[i, :] / s) if s > 0 else (1.0 / N_LEVELS)

    TB[ABSORBING-1, :] = 0.0
    TB[ABSORBING-1, ABSORBING-1] = 1.0
    return TB

def bellman_value_iteration_time_cost(TA: np.ndarray, TB: np.ndarray, cost_hours: np.ndarray,
                                      max_iter=20000, tol=1e-12):
    """
    Bellman en heures :
      V(5)=0
      V(s)=c(s) + min_a sum_{s'} P(s'|s,a) V(s')
    cost_hours: length 5, cost_hours[4]=0
    """
    V = np.zeros(N_LEVELS, dtype=float)
    V[:4] = float(np.max(cost_hours[:4])) * 5.0
    pol = {s: A for s in TRANSIENT}

    for _ in range(max_iter):
        V_old = V.copy()
        for s in TRANSIENT:
            c = float(cost_hours[s-1])
            qA = c + float(np.dot(TA[s-1, :], V_old))
            qB = c + float(np.dot(TB[s-1, :], V_old))
            if qB < qA:
                V[s-1] = qB
                pol[s] = B
            else:
                V[s-1] = qA
                pol[s] = A
        V[ABSORBING-1] = 0.0
        if np.max(np.abs(V - V_old)) < tol:
            break
    return V, pol

def choose_action(level: int, policy: dict) -> str:
    return policy.get(int(level), A)

def next_state_sample(level: int, action: str, TA: np.ndarray, TB: np.ndarray, rng: np.random.Generator) -> int:
    row = TA[level-1, :] if action == A else TB[level-1, :]
    row = np.clip(row, 0.0, 1.0)
    s = float(row.sum())
    row = (row / s) if s > EPS else (np.ones_like(row) / len(row))
    return int(rng.choice(np.arange(1, N_LEVELS+1), p=row))

def next_state_most_likely(level: int, action: str, TA: np.ndarray, TB: np.ndarray) -> int:
    row = TA[level-1, :] if action == A else TB[level-1, :]
    return int(np.argmax(row) + 1)

# ------------------------------------------------------------
# FALLBACK (AUTO) — activities_for_level / action_to_sequence
#   Priorité: L1 (séquences journalières pos1..pos4)
# ------------------------------------------------------------
L1_PATH = globals().get("L1_PATH", "./data/2025-08/L1.20250818-DataMathsElysa.xlsx")

def _detect_sequence_sheet(xlsx_path: str) -> str:
    xl = pd.ExcelFile(xlsx_path)
    for sh in xl.sheet_names:
        tmp = pd.read_excel(xlsx_path, sheet_name=sh, nrows=3)
        cols = [str(c).strip().lower() for c in tmp.columns]
        if any("semaine" in c for c in cols) and any("jour" in c for c in cols):
            return sh
    return xl.sheet_names[0]

def _build_seq_type_from_rows(df_seq: pd.DataFrame, activity_cols):
    # pour chaque position 1..4: activité la plus fréquente
    pos_buckets = {1: [], 2: [], 3: [], 4: []}
    for _, r in df_seq.iterrows():
        for col in activity_cols:
            v = r.get(col, np.nan)
            if pd.isna(v):
                continue
            try:
                iv = int(v)
            except Exception:
                continue
            if 1 <= iv <= 4:
                pos_buckets[iv].append(col)

    seq = {}
    for pos in [1, 2, 3, 4]:
        if len(pos_buckets[pos]) == 0:
            seq[f"pos{pos}"] = None
        else:
            seq[f"pos{pos}"] = pd.Series(pos_buckets[pos]).mode().iloc[0]
    return seq

LEVEL_SEQS = None

if ("activities_for_level" not in globals()) or ("action_to_sequence" not in globals()):
    if os.path.exists(L1_PATH):
        try:
            sh = _detect_sequence_sheet(L1_PATH)
            d = pd.read_excel(L1_PATH, sheet_name=sh)

            # colonnes meta
            lower = {c: str(c).strip().lower() for c in d.columns}
            meta_cols = [c for c, lc in lower.items() if ("semaine" in lc or "jour" in lc)]
            activity_cols = [c for c in d.columns if c not in meta_cols]

            # si une colonne "Level/Niveau/Pretest" existe, on fait par niveau, sinon global
            level_col = None
            for c in d.columns:
                lc = str(c).strip().lower()
                if lc in ["level", "niveau", "pretest", "pretest_i", "niveau_pretest"]:
                    level_col = c
                    break

            LEVEL_SEQS = {}

            if level_col is None:
                seq_global = _build_seq_type_from_rows(d, activity_cols)
                for lvl in LEVELS:
                    LEVEL_SEQS[int(lvl)] = {
                        "activities": list(activity_cols),
                        "seqA": dict(seq_global),
                        "seqB": dict(seq_global),  # A/B = vitesse, même séquence par défaut
                    }
            else:
                for lvl, g in d.groupby(level_col):
                    try:
                        lv = int(lvl)
                    except Exception:
                        continue
                    lv = max(1, min(5, lv))
                    seq_lv = _build_seq_type_from_rows(g, activity_cols)
                    LEVEL_SEQS[lv] = {
                        "activities": list(activity_cols),
                        "seqA": dict(seq_lv),
                        "seqB": dict(seq_lv),
                    }
                # compléter niveaux manquants avec global si besoin
                if len(LEVEL_SEQS) < 5:
                    seq_global = _build_seq_type_from_rows(d, activity_cols)
                    for lvl in LEVELS:
                        if lvl not in LEVEL_SEQS:
                            LEVEL_SEQS[int(lvl)] = {
                                "activities": list(activity_cols),
                                "seqA": dict(seq_global),
                                "seqB": dict(seq_global),
                            }

            print("[INFO] Fallback activités/séquences construit depuis L1:", L1_PATH, "| sheet:", sh)

        except Exception as e:
            LEVEL_SEQS = None
            print("[WARN] Fallback L1 impossible:", str(e))
    else:
        print("[WARN] L1 introuvable -> fallback minimal (pas de vraie séquence).")

    # définir les fonctions si elles n'existent pas
    if "activities_for_level" not in globals():
        def activities_for_level(level: int):
            if LEVEL_SEQS is None:
                raise NameError("activities_for_level(level) indisponible: L1 non exploitable.")
            return list(LEVEL_SEQS[int(level)]["activities"])
        print("[INFO] activities_for_level(level) défini (fallback).")

    if "action_to_sequence" not in globals():
        def action_to_sequence(level: int, action: str):
            if LEVEL_SEQS is None:
                raise NameError("action_to_sequence(level, action) indisponible: L1 non exploitable.")
            pack = LEVEL_SEQS[int(level)]
            if str(action).upper() == "B":
                return dict(pack["seqB"])
            return dict(pack["seqA"])
        print("[INFO] action_to_sequence(level, action) défini (fallback).")

# ------------------------------------------------------------
# (1) PLAN MDP fully observed — durée variable
# ------------------------------------------------------------
def build_plan_mdp_auto_stop(start_level: int,
                            policy: dict,
                            TA: np.ndarray, TB: np.ndarray,
                            mode: str = "most_likely",   # "most_likely" or "sample"
                            seed: int = 42,
                            stop_at: int = 5,
                            max_days: int = MAX_DAYS_CAP):
    rng = np.random.default_rng(seed)
    current = int(start_level)
    plan = []

    for day in range(1, max_days+1):
        if current >= stop_at:
            break

        action = choose_action(current, policy)

        # ---- dépendances  (doivent exister) ----
        if "action_to_sequence" not in globals():
            raise NameError("action_to_sequence(level, action) n'existe pas dans le namespace. "
                            "Exécute d'abord ton code principal (partie activités).")
        seq = action_to_sequence(current, action)

        plan.append({
            "Day": day,
            "Level": current,
            "Action": action,
            "pos1": seq.get("pos1"),
            "pos2": seq.get("pos2"),
            "pos3": seq.get("pos3"),
            "pos4": seq.get("pos4"),
        })

        if mode == "sample":
            current = next_state_sample(current, action, TA, TB, rng)
        else:
            current = next_state_most_likely(current, action, TA, TB)

    reached = (current >= stop_at)
    return plan, reached, current

# ------------------------------------------------------------
# (2) PRINT + SAVE
# ------------------------------------------------------------
def print_and_save_plan_variable_days(plan_rows,
                                      technique: str,
                                      by_model: str,
                                      start_level: int,
                                      target_level: int,
                                      filename_prefix: str,
                                      mode_label: str):
    X = len(plan_rows)
    title = (f"Best sequences of activities selected for {X} days (auto-stop) "
             f"(Start {start_level}, Target {target_level}) :  {technique} |  byModel={by_model} |  mode={mode_label}")
    print(title)

    if "activities_for_level" not in globals():
        raise NameError("activities_for_level(level) n'existe pas dans le namespace. "
                        "Exécute d'abord ton code principal (partie activités).")

    base_acts = activities_for_level(start_level)

    lines = [title]
    out_rows = []

    for i, r in enumerate(plan_rows, start=1):
        seq = {"pos1": r.get("pos1"), "pos2": r.get("pos2"), "pos3": r.get("pos3"), "pos4": r.get("pos4")}
        v = [0]*len(base_acts)
        for pos in [1,2,3,4]:
            a = seq.get(f"pos{pos}")
            if a in base_acts:
                v[base_acts.index(a)] = pos

        print(f"Day {i}: {v}")
        lines.append(f"Day {i}: {v}")

        out_rows.append({
            "Day": i,
            "StartLevel": start_level,
            "TargetLevel": target_level,
            "Technique": technique,
            "ByModel": by_model,
            "Mode": mode_label,
            "LevelUsedForDecision": r.get("Level", ""),
            "Action": r.get("Action", ""),
            "pos1": seq.get("pos1",""),
            "pos2": seq.get("pos2",""),
            "pos3": seq.get("pos3",""),
            "pos4": seq.get("pos4",""),
            "HoursPerDay": HOURS_PER_DAY,
            "CumHoursIfCompleted": round(i * HOURS_PER_DAY, 2),
        })

    safe_model = _safe_tag(by_model)
    out_txt = out_path(f"{filename_prefix}_autoStop_Start{start_level}_Target{target_level}__byModel-{safe_model}__{mode_label}.txt")
    out_csv = out_path(f"{filename_prefix}_autoStop_Start{start_level}_Target{target_level}__byModel-{safe_model}__{mode_label}.csv")

    with open(out_txt, "w", encoding="utf-8") as f:
        f.write("\n".join(lines) + "\n")
        f.write("\n(0 = non sélectionnée ce jour ; 1..4 = position de l’activité)\n")

    pd.DataFrame(out_rows).to_csv(out_csv, index=False, encoding="utf-8-sig")
    return out_txt, out_csv, X

# ------------------------------------------------------------
# (3) POMDP auto-stop (si variables présentes)
# ------------------------------------------------------------
def normalize(p: np.ndarray) -> np.ndarray:
    s = float(p.sum())
    if s <= 0:
        return np.ones_like(p) / len(p)
    return p / s

def belief_init_from_observation(obs_level: int, O_emit: np.ndarray, prior=None) -> np.ndarray:
    if prior is None:
        prior = np.ones(O_emit.shape[1], dtype=float) / O_emit.shape[1]
    b = prior * O_emit[obs_level-1, :]
    return normalize(b)

def belief_predict(b: np.ndarray, Th: np.ndarray) -> np.ndarray:
    return normalize(b.dot(Th))

def belief_update(b_pred: np.ndarray, obs_level: int, O_emit: np.ndarray) -> np.ndarray:
    b_new = b_pred * O_emit[obs_level-1, :]
    return normalize(b_new)

def most_likely_observation_from_belief(b: np.ndarray, O_emit: np.ndarray) -> int:
    p_obs = O_emit.dot(b)
    return int(np.argmax(p_obs) + 1)

def build_plan_pomdp_auto_stop(start_obs_level: int,
                               O_emit: np.ndarray,
                               T_hidden_A: np.ndarray,
                               T_hidden_B: np.ndarray,
                               pi_hidden_star: dict,   # keys: 0/1
                               hidden_names=("Faible","Maitrise"),
                               mode_obs: str = "most_likely",
                               seed: int = 42,
                               max_days: int = MAX_DAYS_CAP):
    rng = np.random.default_rng(seed)
    H0, H1 = 0, 1

    b = belief_init_from_observation(int(start_obs_level), O_emit)
    plan = []
    obs_level = int(start_obs_level)

    for day in range(1, max_days+1):
        z_hat = int(np.argmax(b))
        if z_hat == H1:
            break

        action = pi_hidden_star.get(z_hat, A)

        if "action_to_sequence" not in globals():
            raise NameError("action_to_sequence(level, action) n'existe pas (requis pour POMDP aussi).")
        seq = action_to_sequence(obs_level, action)

        plan.append({
            "Day": day,
            "ObsLevel": obs_level,
            "z_hat": hidden_names[z_hat],
            "belief_Faible": float(b[H0]),
            "belief_Maitrise": float(b[H1]),
            "Action": action,
            "pos1": seq.get("pos1"),
            "pos2": seq.get("pos2"),
            "pos3": seq.get("pos3"),
            "pos4": seq.get("pos4"),
        })

        Th = T_hidden_A if action == A else T_hidden_B
        b_pred = belief_predict(b, Th)

        if mode_obs == "sample":
            p_obs = O_emit.dot(b_pred)
            p_obs = p_obs / max(EPS, float(p_obs.sum()))
            obs_level = int(rng.choice(np.arange(1, N_LEVELS+1), p=p_obs))
        else:
            obs_level = most_likely_observation_from_belief(b_pred, O_emit)

        b = belief_update(b_pred, obs_level, O_emit)

    reached = (int(np.argmax(b)) == H1)
    return plan, reached

def print_and_save_plan_pomdp_variable(plan_rows,
                                       technique: str,
                                       by_model: str,
                                       start_obs_level: int,
                                       target_level: int,
                                       filename_prefix: str,
                                       mode_label: str):
    X = len(plan_rows)
    title = (f"Best sequences of activities selected for {X} days (auto-stop) "
             f"(Start {start_obs_level}, Target {target_level}) :  {technique} |  byModel={by_model} |  mode={mode_label}")
    print(title)

    if "activities_for_level" not in globals():
        raise NameError("activities_for_level(level) n'existe pas (requis pour l'affichage vecteur).")

    base_acts = activities_for_level(start_obs_level)

    lines = [title]
    out_rows = []

    for i, r in enumerate(plan_rows, start=1):
        seq = {"pos1": r.get("pos1"), "pos2": r.get("pos2"), "pos3": r.get("pos3"), "pos4": r.get("pos4")}
        v = [0]*len(base_acts)
        for pos in [1,2,3,4]:
            a = seq.get(f"pos{pos}")
            if a in base_acts:
                v[base_acts.index(a)] = pos

        print(f"Day {i}: {v}")
        lines.append(f"Day {i}: {v}")

        out_rows.append({
            "Day": i,
            "StartObsLevel": start_obs_level,
            "TargetLevel": target_level,
            "Technique": technique,
            "ByModel": by_model,
            "Mode": mode_label,
            "ObsLevel": r.get("ObsLevel",""),
            "z_hat": r.get("z_hat",""),
            "belief_Faible": r.get("belief_Faible",""),
            "belief_Maitrise": r.get("belief_Maitrise",""),
            "Action": r.get("Action",""),
            "pos1": seq.get("pos1",""),
            "pos2": seq.get("pos2",""),
            "pos3": seq.get("pos3",""),
            "pos4": seq.get("pos4",""),
            "HoursPerDay": HOURS_PER_DAY,
            "CumHoursIfCompleted": round(i * HOURS_PER_DAY, 2),
        })

    safe_model = _safe_tag(by_model)
    out_txt = out_path(f"{filename_prefix}_autoStop_Start{start_obs_level}_Target{target_level}__byModel-{safe_model}__{mode_label}.txt")
    out_csv = out_path(f"{filename_prefix}_autoStop_Start{start_obs_level}_Target{target_level}__byModel-{safe_model}__{mode_label}.csv")

    with open(out_txt, "w", encoding="utf-8") as f:
        f.write("\n".join(lines) + "\n")
        f.write("\n(0 = non sélectionnée ce jour ; 1..4 = position de l’activité)\n")

    pd.DataFrame(out_rows).to_csv(out_csv, index=False, encoding="utf-8-sig")
    return out_txt, out_csv, X

# ------------------------------------------------------------
# (4) CHARGEMENTS / POLITIQUES (π*_hours / π_ML_state)
# ------------------------------------------------------------
print("\n==================== Section 1 — AUTO-STOP PLANNING ====================")
print(f"OUT_DIR = {OUT_DIR}")
print(f"START_LEVEL={START_LEVEL}, TARGET_LEVEL={TARGET_LEVEL}, MAX_DAYS_CAP={MAX_DAYS_CAP}")

df = pd.read_csv(L3_PATH)
if "Pretest_i" not in df.columns and "Pretest" in df.columns:
    df["Pretest_i"] = df["Pretest"].apply(cap_level)

# T_A/T_B
if os.path.exists(TA_CSV) and os.path.exists(TB_CSV):
    T_A = load_T_from_csv(TA_CSV)
    T_B = load_T_from_csv(TB_CSV)
    print("[INFO] T_A/T_B chargées depuis L4 CSV.")
else:
    print("[INFO] L4 CSV introuvables -> reconstruction T_A/T_B depuis L3.")
    T_A = empirical_transition_matrix_from_pretest_final(df)
    T_B = make_intensive_matrix_from_standard(T_A)

# Politique optimisée temps: priorité à SECTION2 JSON si dispo
pi_hours_star = None
if os.path.exists(SECTION2_JSON):
    try:
        with open(SECTION2_JSON, "r", encoding="utf-8") as f:
            SECTION2 = json.load(f)
        if isinstance(SECTION2.get("pi_hours_star", None), dict):
            # clés potentiellement string -> cast int
            pi_hours_star = {int(k): v for k, v in SECTION2["pi_hours_star"].items()}
            print("[INFO] pi_hours_star chargée depuis Section 1 JSON.")
    except Exception as e:
        print("[WARN] Lecture SECTION2 JSON impossible:", str(e))

# Si pas trouvé, calcule Bellman temps avec coût simple 8h/étape
if pi_hours_star is None:
    print("[INFO] pi_hours_star non trouvée -> calcul Bellman temps avec coût 8h/étape (fallback).")
    cost_hours = np.array([HOURS_PER_DAY]*4 + [0.0], dtype=float)
    _, pi_hours_star = bellman_value_iteration_time_cost(T_A, T_B, cost_hours)

# π_ML_state si présent dans SECTION2 JSON
pi_ml_state = None
if os.path.exists(SECTION2_JSON):
    try:
        if SECTION2.get("pi_ml_state", None) is not None:
            pi_ml_state = {int(k): v for k, v in SECTION2["pi_ml_state"].items()}
            print("[INFO] pi_ml_state chargée depuis Section 1 JSON.")
    except Exception:
        pass

print("\n--- Politiques disponibles ---")
print("π*_hours:", pi_hours_star)
print("π_ML_state:", pi_ml_state)

# ------------------------------------------------------------
# (5) RUN MDP auto-stop
# ------------------------------------------------------------
policy_opt = pi_hours_star

# most_likely
plan_opt_ml, reached_opt, final_state_opt = build_plan_mdp_auto_stop(
    start_level=START_LEVEL,
    policy=policy_opt,
    TA=T_A, TB=T_B,
    mode="most_likely",
    seed=42,
    stop_at=ABSORBING,
    max_days=MAX_DAYS_CAP
)

print("\nBaseline MDP (fully observed) — POLICY=π*_hours (auto-stop) :\n")
opt_txt, opt_csv, opt_X = print_and_save_plan_variable_days(
    plan_opt_ml,
    technique=TECHNIQUE_MDP,
    by_model=BY_MODEL_NAME,
    start_level=START_LEVEL,
    target_level=ABSORBING,
    filename_prefix="best_sequences_MDP_OPT",
    mode_label="most_likely"
)

print(f"\n[INFO] Reached absorption? {reached_opt} | final_state={final_state_opt} | "
      f"X_days={opt_X} | approx_total_hours={opt_X*HOURS_PER_DAY:.1f}h")

# sample
plan_opt_sample, reached_opt_s, final_state_opt_s = build_plan_mdp_auto_stop(
    start_level=START_LEVEL,
    policy=policy_opt,
    TA=T_A, TB=T_B,
    mode="sample",
    seed=42,
    stop_at=ABSORBING,
    max_days=MAX_DAYS_CAP
)

print("\nBaseline MDP (fully observed) — POLICY=π*_hours (auto-stop, sample) :\n")
optS_txt, optS_csv, optS_X = print_and_save_plan_variable_days(
    plan_opt_sample,
    technique=TECHNIQUE_MDP,
    by_model=BY_MODEL_NAME,
    start_level=START_LEVEL,
    target_level=ABSORBING,
    filename_prefix="best_sequences_MDP_OPT",
    mode_label="sample_seed42"
)

print(f"\n[INFO] (sample) Reached absorption? {reached_opt_s} | final_state={final_state_opt_s} | "
      f"X_days={optS_X} | approx_total_hours={optS_X*HOURS_PER_DAY:.1f}h")

# MDP sous π_ML_state (si dispo)
ml_txt = ml_csv = None
ml_X = None
reached_ml = None
final_state_ml = None

if isinstance(pi_ml_state, dict):
    plan_ml, reached_ml, final_state_ml = build_plan_mdp_auto_stop(
        start_level=START_LEVEL,
        policy=pi_ml_state,
        TA=T_A, TB=T_B,
        mode="most_likely",
        seed=42,
        stop_at=ABSORBING,
        max_days=MAX_DAYS_CAP
    )

    print("\nBaseline MDP (fully observed) — POLICY=π_ML_state (auto-stop) :\n")
    ml_txt, ml_csv, ml_X = print_and_save_plan_variable_days(
        plan_ml,
        technique=TECHNIQUE_MDP,
        by_model=BY_MODEL_NAME,
        start_level=START_LEVEL,
        target_level=ABSORBING,
        filename_prefix="best_sequences_MDP_piML",
        mode_label="most_likely"
    )
    print(f"\n[INFO] π_ML_state Reached absorption? {reached_ml} | final_state={final_state_ml} | "
          f"X_days={ml_X} | approx_total_hours={ml_X*HOURS_PER_DAY:.1f}h")
else:
    print("\n[INFO] pi_ml_state non disponible (exécute Section 1+5 pour l’obtenir).")

# ------------------------------------------------------------
# (6) RUN POMDP auto-stop
# ------------------------------------------------------------
p_txt = p_csv = None
p_X = None
reached_p = None

if all(k in globals() for k in ["O_emit", "T_hidden_A", "T_hidden_B", "pi_hidden_star"]):
    # pi_hidden_star peut être dict int->A/B, sinon on tente de le convertir
    if isinstance(pi_hidden_star, dict) and len(pi_hidden_star) > 0:
        any_key = list(pi_hidden_star.keys())[0]
        if not isinstance(any_key, int):
            # cas où clés sont "Faible"/"Maitrise"
            pi_hidden_star_int = {0: pi_hidden_star.get("Faible", A), 1: pi_hidden_star.get("Maitrise", A)}
        else:
            pi_hidden_star_int = pi_hidden_star
    else:
        pi_hidden_star_int = {0: A, 1: A}

    plan_pomdp_ml, reached_p = build_plan_pomdp_auto_stop(
        start_obs_level=START_LEVEL,
        O_emit=O_emit,
        T_hidden_A=T_hidden_A,
        T_hidden_B=T_hidden_B,
        pi_hidden_star=pi_hidden_star_int,
        hidden_names=("Faible", "Maitrise"),
        mode_obs="most_likely",
        seed=42,
        max_days=MAX_DAYS_CAP
    )

    print("\nPOMDP approx (HMM belief + projection) — POLICY=π_hidden* (auto-stop) :\n")
    p_txt, p_csv, p_X = print_and_save_plan_pomdp_variable(
        plan_pomdp_ml,
        technique=TECHNIQUE_POMDP,
        by_model=BY_MODEL_NAME,
        start_obs_level=START_LEVEL,
        target_level=ABSORBING,
        filename_prefix="best_sequences_POMDP_OPT",
        mode_label="most_likely"
    )
    print(f"\n[INFO] POMDP Reached latent mastery? {reached_p} | X_days={p_X} | approx_total_hours={p_X*HOURS_PER_DAY:.1f}h")
else:
    print("\n[INFO] POMDP non exécuté : variables manquantes (O_emit/T_hidden_A/T_hidden_B/pi_hidden_star).")

# ------------------------------------------------------------
# (7) Export résumé
# ------------------------------------------------------------
summary6 = {
    "OUT_DIR": OUT_DIR,
    "MAX_DAYS_CAP": MAX_DAYS_CAP,
    "HOURS_PER_DAY": HOURS_PER_DAY,
    "START_LEVEL": START_LEVEL,
    "TARGET_LEVEL": TARGET_LEVEL,
    "pi_hours_star": pi_hours_star,
    "pi_ml_state": (pi_ml_state if isinstance(pi_ml_state, dict) else None),

    "MDP_OPT_most_likely": {
        "X_days": opt_X, "total_hours": float(opt_X * HOURS_PER_DAY),
        "reached": bool(reached_opt), "final_state": int(final_state_opt),
        "txt": opt_txt, "csv": opt_csv
    },
    "MDP_OPT_sample_seed42": {
        "X_days": optS_X, "total_hours": float(optS_X * HOURS_PER_DAY),
        "reached": bool(reached_opt_s), "final_state": int(final_state_opt_s),
        "txt": optS_txt, "csv": optS_csv
    },
    "MDP_piML_most_likely": (
        {
            "X_days": ml_X, "total_hours": float(ml_X * HOURS_PER_DAY),
            "reached": bool(reached_ml), "final_state": int(final_state_ml),
            "txt": ml_txt, "csv": ml_csv
        } if ml_X is not None else None
    ),
    "POMDP_OPT_most_likely": (
        {
            "X_days": p_X, "total_hours": float(p_X * HOURS_PER_DAY),
            "reached": bool(reached_p),
            "txt": p_txt, "csv": p_csv
        } if p_X is not None else None
    )
}

with open(out_path("SECTION2_autostop_summary.json"), "w", encoding="utf-8") as f:
    json.dump(summary6, f, ensure_ascii=False, indent=2)

print("\n Export Section 1:", out_path("SECTION1_autostop_summary.json"))

print("\n==================== INTERPRÉTATION  ====================")
print("1) Le plan 'auto-stop' s’arrête automatiquement quand le niveau 5 est atteint.")
print("2) X_days est donc le nombre de jours (séquences journalières) nécessaires selon la politique.")
print("3) Total_hours ≈ X_days * 8h (car 4 activités/jour * 2h/activité).")
print("4) 'most_likely' donne un chemin typique (argmax). 'sample' donne un scénario plausible (stochastique).")
print("5) Si π*_hours produit un X_days plus petit que π_A (standard), l’action B (intensive) apporte un gain de temps.")


[INFO] Fallback activités/séquences construit depuis L1: ./data/2025-08/L1.20250818-DataMathsElysa.xlsx | sheet: Activités
[INFO] activities_for_level(level) défini (fallback).
[INFO] action_to_sequence(level, action) défini (fallback).

==================== PAGE 6 — AUTO-STOP PLANNING ====================
OUT_DIR = ./out/202602/out_pomdp
START_LEVEL=1, TARGET_LEVEL=5, MAX_DAYS_CAP=60
[INFO] L4 CSV introuvables -> reconstruction T_A/T_B depuis L3.
[INFO] pi_hours_star chargée depuis Page 5 JSON.

--- Politiques disponibles ---
π*_hours: {1: 'B', 2: 'B', 3: 'B', 4: 'B'}
π_ML_state: None

Baseline MDP (fully observed) — POLICY=π*_hours (auto-stop) :

Best sequences of activities selected for 60 days (auto-stop) (Start 1, Target 5) :  Baseline MDP (fully observed) |  byModel=PolicyTimeOptimal |  mode=most_likely
Day 1: [0, 0, 0, 0, 0, 2, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0]
Day 2: [0, 0, 0

### Section 1 — Analyse & validation de l’hypothèse (A vs B selon le niveau)

In [ ]:
# -*- coding: utf-8 -*-
"""
Section 1 — Analyse & validation de l’hypothèse (A vs B selon le niveau)
+ Synthèses  + exports CSV/JSON

Hypothèse à tester (non vérité absolue) :
- niveaux 1–2 : action A plus adaptée
- niveaux 3–4 : action B plus adaptée

Ce que cette page fait concrètement :
1) Compare A vs B par NIVEAU via 3 angles complémentaires :
   (i)  MDP en HEURES : Q_hours(s,a) = hours_per_day + sum_{s'} P(s'|s,a) * V_hours*(s')
        -> action “meilleure” = argmin Q_hours
   (ii) Monte-Carlo “temps d’absorption” en heures : simulation stochastique jusqu’à 5
        -> compare moyenne/mediane/quantiles
   (iii) Données réelles (HoursTotal) : compare la durée observée (heures) des élèves
        de chaque niveau, en reliant à la “politique observée” si tu as un label action
        (sinon, on fait une analyse descriptive globale par niveau)

2) Analyse “π_ML” (si disponible) :
   - distribution des actions prédites par niveau (combien A vs B)
   - si tu as proba, moyenne proba(B) par niveau
"""
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import json
import numpy as np
import pandas as pd

EPS = 1e-12
A, B = "A", "B"
LEVELS = [1,2,3,4,5]
TRANSIENT = [1,2,3,4]
ABSORBING = 5
N_LEVELS = 5

#  : 4 activités/jour, 2h/activité
HOURS_PER_ACTIVITY = 2.0
ACTIVITIES_PER_DAY = 4.0
HOURS_PER_DAY = HOURS_PER_ACTIVITY * ACTIVITIES_PER_DAY  # 8h

# Monte-Carlo
MC_N = 2000
MC_MAX_DAYS = 120
MC_SEED = 42

# ------------------------------------------------------------
# (0) Helpers
# ------------------------------------------------------------
def _ensure_prob_row(row: np.ndarray) -> np.ndarray:
    row = np.array(row, dtype=float)
    row = np.clip(row, 0.0, 1.0)
    s = float(row.sum())
    if s <= EPS:
        return np.ones_like(row) / len(row)
    return row / s

def q_hours(level: int, action: str, V_hours: np.ndarray, TA: np.ndarray, TB: np.ndarray) -> float:
    """
    Q_hours(s,a) = cost(hours_per_day) + sum_{s'} P(s'|s,a) * V_hours(s')
    V_hours(5)=0
    """
    row = TA[level-1, :] if action == A else TB[level-1, :]
    row = _ensure_prob_row(row)
    return float(HOURS_PER_DAY + np.dot(row, V_hours))

def pick_best_action_by_q(level: int, V_hours: np.ndarray, TA: np.ndarray, TB: np.ndarray) -> dict:
    qA = q_hours(level, A, V_hours, TA, TB)
    qB = q_hours(level, B, V_hours, TA, TB)
    best = A if qA <= qB else B
    return {"qA_hours": qA, "qB_hours": qB, "best_action": best}

def mc_absorption_time_hours(start_level: int, action_fixed: str, TA: np.ndarray, TB: np.ndarray,
                             n: int = 1000, seed: int = 42, max_days: int = 120) -> np.ndarray:
    """
    Simule n trajectoires jusqu’à absorption (5), en appliquant action_fixed à chaque étape.
    Retourne un vecteur des temps (heures).
    """
    rng = np.random.default_rng(seed)
    times = []

    for _ in range(n):
        s = int(start_level)
        days = 0
        while s != ABSORBING and days < max_days:
            row = TA[s-1, :] if action_fixed == A else TB[s-1, :]
            row = _ensure_prob_row(row)
            s = int(rng.choice(np.arange(1, N_LEVELS+1), p=row))
            days += 1

        times.append(float(days * HOURS_PER_DAY))

    return np.array(times, dtype=float)

def summarize_mc(x: np.ndarray) -> dict:
    x = np.array(x, dtype=float)
    return {
        "mean": float(np.mean(x)),
        "median": float(np.median(x)),
        "p10": float(np.quantile(x, 0.10)),
        "p90": float(np.quantile(x, 0.90)),
        "min": float(np.min(x)),
        "max": float(np.max(x)),
    }

# ------------------------------------------------------------
# (1) Choisir V_hours à utiliser
# ------------------------------------------------------------
if "V_hours_star" in globals():
    Vh = np.array(V_hours_star, dtype=float)
    used_V_name = "V_hours_star"
elif "V_star" in globals():
    # fallback : V_star est en “pas” (jours). On approx en heures : pas * 8h
    Vh = np.array(V_star, dtype=float) * HOURS_PER_DAY
    used_V_name = "V_star_scaled_to_hours"
else:
    # dernier recours
    Vh = np.zeros(N_LEVELS, dtype=float)
    used_V_name = "none"

# ------------------------------------------------------------
# (2) Hypothesis check — via Q_hours (Bellman local)
# ------------------------------------------------------------
rows = []
for s in TRANSIENT:
    d = pick_best_action_by_q(s, Vh, T_A, T_B)
    rows.append({
        "Level": s,
        "BestAction_by_Qhours": d["best_action"],
        "Q_A_hours": round(d["qA_hours"], 3),
        "Q_B_hours": round(d["qB_hours"], 3),
        "Delta_B_minus_A_hours": round(d["qB_hours"] - d["qA_hours"], 3),  # <0 => B better
    })

df_q = pd.DataFrame(rows).sort_values("Level")
df_q.to_csv(out_path("SECTION1_hypothesis_level_summary.csv"), index=False, encoding="utf-8-sig")

print("\n==================== Section 1 — Hypothesis (Bellman/Q in hours) ====================")
print(f"[INFO] Value used: {used_V_name}")
print(df_q.to_string(index=False))
print("\nLecture  rapide :")
print("- Delta_B_minus_A_hours < 0  => B réduit l’espérance de temps (meilleur).")
print("- Delta_B_minus_A_hours > 0  => A est meilleur.")
print("=> Compare niveaux 1–2 vs 3–4 pour confirmer/infirmer l’hypothèse.\n")

# ------------------------------------------------------------
# (3) Hypothesis check — Monte-Carlo (A fixe vs B fixe)
# ------------------------------------------------------------
mc_rows = []
for s in TRANSIENT:
    tA = mc_absorption_time_hours(s, A, T_A, T_B, n=MC_N, seed=MC_SEED+s, max_days=MC_MAX_DAYS)
    tB = mc_absorption_time_hours(s, B, T_A, T_B, n=MC_N, seed=MC_SEED+100+s, max_days=MC_MAX_DAYS)

    sA = summarize_mc(tA)
    sB = summarize_mc(tB)

    mc_rows.append({
        "Level": s,
        "MC_N": MC_N,
        "A_mean_h": round(sA["mean"], 2),
        "B_mean_h": round(sB["mean"], 2),
        "A_median_h": round(sA["median"], 2),
        "B_median_h": round(sB["median"], 2),
        "B_minus_A_mean_h": round(sB["mean"] - sA["mean"], 2),   # <0 => B better
        "B_minus_A_median_h": round(sB["median"] - sA["median"], 2),
        "A_p10_h": round(sA["p10"], 2),
        "A_p90_h": round(sA["p90"], 2),
        "B_p10_h": round(sB["p10"], 2),
        "B_p90_h": round(sB["p90"], 2),
    })

df_mc = pd.DataFrame(mc_rows).sort_values("Level")
df_mc.to_csv(out_path("SECTION2_mc_summary.csv"), index=False, encoding="utf-8-sig")

print("\n==================== Section 2 — Hypothesis (Monte-Carlo absorption time) ====================")
print(df_mc.to_string(index=False))
print("\nLecture  rapide :")
print("- B_minus_A_mean_h < 0  => en moyenne, B mène plus vite à la maîtrise (niveau 5).")
print("- Les quantiles p10/p90 donnent l’incertitude (variabilité des trajectoires).\n")

# ------------------------------------------------------------
# (4) Données réelles — HoursTotal par niveau (descriptif)
# ------------------------------------------------------------
df_real = df.dropna(subset=["Pretest_i", "HoursTotal"]).copy()
df_real["Pretest_i"] = df_real["Pretest_i"].astype(int)
df_real["HoursTotal"] = pd.to_numeric(df_real["HoursTotal"], errors="coerce")

real_rows = []
for s in TRANSIENT:
    dfi = df_real[df_real["Pretest_i"] == s]
    if len(dfi) == 0:
        continue
    x = dfi["HoursTotal"].dropna().values.astype(float)
    if len(x) == 0:
        continue
    real_rows.append({
        "Level": s,
        "n_students": int(len(x)),
        "HoursTotal_mean": round(float(np.mean(x)), 2),
        "HoursTotal_median": round(float(np.median(x)), 2),
        "HoursTotal_p10": round(float(np.quantile(x, 0.10)), 2),
        "HoursTotal_p90": round(float(np.quantile(x, 0.90)), 2),
        "HoursTotal_min": round(float(np.min(x)), 2),
        "HoursTotal_max": round(float(np.max(x)), 2),
    })

df_real_sum = pd.DataFrame(real_rows).sort_values("Level")
df_real_sum.to_csv(out_path("SECTION2_real_hours_by_level.csv"), index=False, encoding="utf-8-sig")

print("\n==================== Section 2 — Réel (HoursTotal) par niveau ====================")
if len(df_real_sum) > 0:
    print(df_real_sum.to_string(index=False))
    print("\nLecture  :")
    print("- Ceci décrit le temps de remediation OBSERVÉ (heures) selon le niveau initial.")
    print("- Ça ne prouve pas A vs B si on n’a pas de label 'action réelle', mais ça contextualise la difficulté par niveau.\n")
else:
    print("[WARN] Pas de données HoursTotal exploitables.\n")

# ------------------------------------------------------------
# (5) π_ML : distribution des actions prédites par niveau (si dispo)
# ------------------------------------------------------------
ml_exported = False
if "pi_ml_state" in globals() and isinstance(pi_ml_state, dict):
    # pi_ml_state est une “politique stationnaire” (par niveau) => simple tableau
    ml_rows = []
    for s in TRANSIENT:
        ml_rows.append({
            "Level": s,
            "pi_ml_state_action": pi_ml_state.get(s, A)
        })
    df_ml = pd.DataFrame(ml_rows)
    df_ml.to_csv(out_path("SECTION2_ml_policy_by_level.csv"), index=False, encoding="utf-8-sig")
    ml_exported = True

    print("\n==================== Section 3 — π_ML_state (stationnaire) ====================")
    print(df_ml.to_string(index=False))
    print("\nLecture  :")
    print("- π_ML_state dit : 'si un élève est au niveau s, l’action la plus probable observée/prévue est A ou B'.\n")

# Si tu as une fonction de proba par ligne (Section 1), on peut aussi faire : moyenne P(B|features) par niveau
if "predict_action_proba_row" in globals() and callable(predict_action_proba_row):
    dfx = df.dropna(subset=["Pretest_i"]).copy()
    dfx["Pretest_i"] = dfx["Pretest_i"].astype(int)

    prows = []
    for s in TRANSIENT:
        dfi = dfx[dfx["Pretest_i"] == s].head(500).copy()  # cap sécurité
        if len(dfi) == 0:
            continue
        pB_list = []
        for _, r in dfi.iterrows():
            pr = predict_action_proba_row(r)  # expected dict or tuple
            # on accepte plusieurs formats robustes :
            if isinstance(pr, dict) and "pB" in pr:
                pB_list.append(float(pr["pB"]))
            elif isinstance(pr, (list, tuple)) and len(pr) >= 1:
                pB_list.append(float(pr[0]))
        if len(pB_list) == 0:
            continue

        prows.append({
            "Level": s,
            "n_rows_used": int(len(pB_list)),
            "mean_pB": round(float(np.mean(pB_list)), 4),
            "median_pB": round(float(np.median(pB_list)), 4),
        })

    if len(prows) > 0:
        df_pB = pd.DataFrame(prows).sort_values("Level")
        df_pB.to_csv(out_path("SECTION2_ml_probB_by_level.csv"), index=False, encoding="utf-8-sig")

        print("\n==================== Section 3 — π_ML(features): proba(B) par niveau ====================")
        print(df_pB.to_string(index=False))
        print("\nLecture  :")
        print("- mean_pB proche de 1 => le ML pense que B est souvent adapté pour ce niveau/ces profils.")
        print("- mean_pB proche de 0 => plutôt A.\n")

# ------------------------------------------------------------
# (6) Verdict automatique sur l’hypothèse (résumé)
# ------------------------------------------------------------
# Hypothèse : 1-2 => A ; 3-4 => B
hyp_ok_q = True
for s in [1,2]:
    act = df_q[df_q["Level"]==s]["BestAction_by_Qhours"].values
    if len(act) and act[0] != A:
        hyp_ok_q = False
for s in [3,4]:
    act = df_q[df_q["Level"]==s]["BestAction_by_Qhours"].values
    if len(act) and act[0] != B:
        hyp_ok_q = False

hyp_ok_mc = True
for s in [1,2]:
    v = df_mc[df_mc["Level"]==s]["B_minus_A_mean_h"].values
    if len(v) and float(v[0]) < 0:  # B faster for level 1/2
        hyp_ok_mc = False
for s in [3,4]:
    v = df_mc[df_mc["Level"]==s]["B_minus_A_mean_h"].values
    if len(v) and float(v[0]) > 0:  # A faster for level 3/4
        hyp_ok_mc = False

verdict = {
    "hypothesis_statement": "Levels 1-2 => A, Levels 3-4 => B",
    "check_Qhours_pass": bool(hyp_ok_q),
    "check_MC_pass": bool(hyp_ok_mc),
    "notes": {
        "Qhours": "best action by minimizing one-step Bellman Q in HOURS",
        "MC": f"mean absorption time (hours) via Monte-Carlo with N={MC_N}"
    }
}

with open(out_path("SECTION2_hypothesis_summary.json"), "w", encoding="utf-8") as f:
    json.dump(verdict, f, ensure_ascii=False, indent=2)

print(json.dumps(verdict, ensure_ascii=False, indent=2))
print("\n Exports Section 1 :")
print("-", out_path("SECTION2_hypothesis_level_summary.csv"))
print("-", out_path("SECTION2_mc_summary.csv"))
print("-", out_path("SECTION2_real_hours_by_level.csv"))
if ml_exported:
    print("-", out_path("SECTION2_ml_policy_by_level.csv"))
print("-", out_path("SECTION2_hypothesis_summary.json"))

print("• Le tableau 'Bellman/Q in hours' compare A vs B pour chaque niveau en termes de temps attendu (heures).")
print("  - Si BestAction_by_Qhours = B pour un niveau, cela signifie que l’intensif réduit l’espérance de temps.")
print("• Le tableau 'Monte-Carlo' simule des trajectoires aléatoires jusqu’au niveau 5 : il montre la vitesse moyenne/typique.")
print("  - B_minus_A_mean_h < 0 signifie que B est plus rapide en moyenne.")
print("• Le tableau 'HoursTotal réel' est un descriptif : il donne la charge horaire observée dans les données par niveau initial.")
print("• La synthèse JSON te dit si l’hypothèse (1–2=>A, 3–4=>B) est confirmée selon ces 2 tests (Qhours et Monte-Carlo).")



==================== PAGE 7 — Hypothesis (Bellman/Q in hours) ====================
[INFO] Value used: V_hours_star
 Level BestAction_by_Qhours  Q_A_hours  Q_B_hours  Delta_B_minus_A_hours
     1                    B     33.298     30.680                 -2.618
     2                    B     30.754     27.830                 -2.924
     3                    B     21.363     18.962                 -2.402
     4                    B     21.530     18.552                 -2.978

Lecture métier rapide :
- Delta_B_minus_A_hours < 0  => B réduit l’espérance de temps (meilleur).
- Delta_B_minus_A_hours > 0  => A est meilleur.
=> Compare niveaux 1–2 vs 3–4 pour confirmer/infirmer l’hypothèse.


==================== PAGE 7 — Hypothesis (Monte-Carlo absorption time) ====================
 Level  MC_N  A_mean_h  B_mean_h  A_median_h  B_median_h  B_minus_A_mean_h  B_minus_A_median_h  A_p10_h  A_p90_h  B_p10_h  B_p90_h
     1  2000     29.82     23.45        24.0        16.0             -6.37      